## Setup

In [ ]:
# Load packages
using DifferentialEquations
using Plots, StatsPlots
using CSV, DataFrames
using Turing
using LaTeXStrings
using XLSX
using Optim
using StatsBase
using Random
using KernelDensity
using ProgressMeter
using Distributions
using Measures
using GLM
using HypothesisTests

# Setup plots standard
Plots.default(fontfamily = ("computer modern"))

# Set seed 
Random.seed!(42)

In [ ]:
# Add rhs file
include("model_rhs.jl")
    
# Add parameter file
include("model_default_param.jl")

# Add treatment rhs
include("model_rhs_treatment.jl")

# Add function for calculating VAF
include("model_calc_VAF.jl")

# Add model for Turing inference - individual
include("model_infer_dynamics.jl")

# Add model for Turing inference - hierarchical - pseudodata
include("model_infer_dynamics_Hierarchical_Gibbs_pseudodata.jl")

In [ ]:
# Setup initial conditions for master curve
x00 = 1.0*10^5
x10 = 2.5*10^6
x20 = 6.4*10^11
y00 = 1
y10 = 0
y20 = 0
a0 = 8.1*10^2
s0 = 1

# Collect in one vector
u0 = [x00,x10,x20,y00,y10,y20,a0,s0]

# Setup and solve ODEproblem
tspan = (0.0,90*365)
prob = ODEProblem(model_rhs, u0, tspan, p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

## Generate pseudodata

In [ ]:
# Set seed 
Random.seed!(42)

# Choose number of patients
P = 25

# Choose initial distributions of rho1 and rho2
rho_bar = 0.04
rho_sd = 0.28
delta_bar = 0.07
delta_sd = 0.31
kL_low = 0.8
kL_high = 1.5
kT_low = 25
kT_high = 100

# Sample from initial distributions
rho_vec = rand(truncated(Normal(rho_bar, rho_sd), lower=0), P)
delta_vec = rand(truncated(Normal(delta_bar, delta_sd), lower=0), P)
initJAK_vec = rand(Uniform(0, 0.99999), P)
kL_vec = rand(Uniform(kL_low, kL_high), P)
kT_vec = rand(Uniform(kT_low, kT_high), P)

# Create pID vector
pID_vec = collect(1:1:P)

# Save in dataframe
df_param = DataFrame(patientID = pID_vec, rho = rho_vec, delta = delta_vec, initJAK = initJAK_vec, kL = kL_vec, kT = kT_vec)

In [ ]:
# Set seed 
Random.seed!(42)

# Choose effect
effect = "py0dy1"

# Choose standard deviation parameter
sigma_v = 0.05
sigma_L = 1*10^9
sigma_T = 2.5*10^10

# Setup data frame for data
df_D = DataFrame(patientID = Int64[], days = Float64[], IFN = Float64[], RUX = Float64[], JAK = Float64[], 
                 JAK_no_noise = Float64[], WBC = Float64[], WBC_no_noise = Float64[], TRC = Float64[], TRC_no_noise = Float64[])

# Generate data from model
for i=1:P
    
    # Define patientID
    pID = i
    
    # Generate number of meaurements
    N = rand(3:8)
    
    # Generate patientID
    pID_vec = pID*ones(Int64, N)
    
    # Generate IFN treatment schedule
    IFN_vec = zeros(N)
    IFN_vec[1] = 45/7
    for j=2:N
        if rand(Uniform(0,1))<0.25
            IFN_vec[j] = 0.5*IFN_vec[j-1]
        else
            IFN_vec[j] = IFN_vec[j-1]
        end
    end
    
    # Generate RUX treatment schedule
    RUX_vec = 0*ones(N)
    
    # Extract rho (rho1) and delta (rho2)
    rho1 = df_param[df_param.patientID .== pID, :].rho[1]
    rho2 = df_param[df_param.patientID .== pID, :].delta[1]
    rho = [rho1,rho2]
    
    # Extract initJAK
    initJAK = df_param[df_param.patientID .== pID, :].initJAK[1]
    
    # Extract kL and kT
    kL = df_param[df_param.patientID .== pID, :].kL[1]
    kT = df_param[df_param.patientID .== pID, :].kT[1]

    # Generate days for treatment
    pDays = rand(DiscreteUniform(1,75*30.4), N-1)
    pDays = vcat(0,sort(pDays))

    # Setup data frame with treatment
    df_p = DataFrame(days = pDays, IFN = IFN_vec, RUX = RUX_vec)

    # Calculate VAF
    VAF, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
   
    # Save all variables
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
    a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

    # Add noise
    VAF_measurements = zeros(N)
    WBC = zeros(N)
    WBC_measurements = zeros(N)
    TRC = zeros(N)
    TRC_measurements = zeros(N)
    for j=1:N
        VAF_measurements[j] = rand(truncated(Normal(VAF[j], sigma_v), lower = 0, upper = 1))
        WBC[j] = kL*(p_curve.x2[j]+p_curve.y2[j])/(30.5*5*10^9)
        WBC_measurements[j] = rand(truncated(Normal(kL*(p_curve.x2[j]+p_curve.y2[j])/(30.5*5), sigma_L), lower = 0))/(10^9)
        TRC[j] = kT*(p_curve.x2[j]+p_curve.y2[j])/(30.5*5*10^9)
        TRC_measurements[j] = rand(truncated(Normal(kT*(p_curve.x2[j]+p_curve.y2[j])/(30.5*5), sigma_T), lower = 0))/(10^9)
    end
    
    # Setup temporary dataframe and save
    df_temp = DataFrame(patientID = pID_vec, days = pDays, IFN = IFN_vec, RUX = RUX_vec, JAK = VAF_measurements, 
                        JAK_no_noise = VAF, WBC = WBC_measurements, WBC_no_noise = WBC, TRC = TRC_measurements, TRC_no_noise =TRC)
    df_D = append!(df_D, df_temp)
end

# Unique patients
unique_patients = unique(df_D.patientID)

df_D

In [ ]:
# Extract data for patient
pID = 1
df_p = df_D[df_D.patientID .== pID, :]

# Extract rho and delta values
rho1 = df_param[df_param.patientID .== pID, :].rho[1]
rho2 = df_param[df_param.patientID .== pID, :].delta[1]
rho = [rho1,rho2]

# Extract initJAK
initJAK = df_param[df_param.patientID .== pID, :].initJAK[1]

# Chose days calculate VAf for
pDays = (0:75*30.4)

# Extract latest data point
maxDays = maximum(df_p.days)

# Calculate VAF
VAF, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)

# Plot results
figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
title!(L"JAK2"*" VAF for Virtual Patient $(pID), \n" *L"\rho="
       *"$(round(rho1; sigdigits=3)), " * L"\delta="*"$(round(rho2; sigdigits=3))")
xlabel!(L"t"*"/months")
ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
xlims!(0, 75)
ylims!(0, 20)
p2 = twinx()
plot!(p2, pDays[pDays.<=maxDays]/30.4, VAF[pDays.<=maxDays,:]*100, linewidth = 6, 
      label = L"JAK2"*" VAF - model", colour = :darkgoldenrod1)
plot!(p2, pDays[pDays.>maxDays]/30.4, VAF[pDays.>maxDays]*100, linewidth = 6, 
      label = L"JAK2"*" VAF - model", colour = :darkgoldenrod1, linestyle = :dash)
scatter!(p2, df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Pseudodata}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
ylabel!(p2, L"JAK2"*" VAF/%")
xlims!(p2, 0, 75)
ylims!(p2, 0, 100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
      margin = 5mm, leftmargin = 8mm, legend = :none)

## Hierarchichal inference procedure

In [ ]:
# Choose treatment effect
effect = "py0dy1"

# Load standard parameters
include("model_default_param.jl");

# Choose all patients for sampling
VAF = df_D.JAK[.!isnan.(df_D.JAK)]

# Instantiate model
model = model_infer_dynamics_Hierarchical_Gibbs_pseudodata(VAF,df_D,effect,p,master_curve)

In [ ]:
space = DynamicPPL.syms(DynamicPPL.VarInfo(model))

In [ ]:
# Set seed 
Random.seed!(42)

# Sample with Turing
n_samples = 10500

# Model symbols
space = DynamicPPL.syms(DynamicPPL.VarInfo(model))

# Number of parameters
N = length(space)

# Initial parameter values for the sampling - from true values
init_cond = [rho_bar, delta_bar, rho_sd, delta_sd]

# Tuple of samplers and initial parameters from true values
A_concrete = ()
for i=1:N
    # Add to sampler list
    A_concrete = (A_concrete..., HMC(0.05, 10, space[i]))
    
    # Add true values as starting point for sampler
    if i>2
        # Add initial parameters from MLE
        temp = collect(df_param[i-2, [:rho, :delta, :initJAK]])
        append!(temp, sigma_v)
        if temp[1]<0.01
            temp[1] = 0.01
        end
        if temp[2]<0.01
            temp[2] = 0.01
        end
        if temp[3]<0.01
            temp[3] = 0.01
        end
        if temp[4]<0.01
            temp[4] = 0.01
        end
        append!(init_cond, temp)
    end
end

# Setup for sampler
A = typeof(A_concrete)
B_concrete = ntuple(i->1, N)
B = typeof(B_concrete)

# Sampler
sampler = Gibbs{space,N,A,B}(A_concrete,B_concrete)

# Sample
MCMC = sample(model, sampler, n_samples; initial_params=init_cond)

In [ ]:
# Convert to dataframe 
df_MCMC = DataFrame(MCMC)

# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Hierarchical/MCMC_py0dy1_Hierarchical_Gibbs_pseudodata.csv"
CSV.write(path, df_MCMC)

## Results for individual patients

In [ ]:
# Setup dataframes
df_MCMC = DataFrame

# Load data
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Hierarchical/MCMC_py0dy1_Hierarchical_Gibbs_pseudodata.csv"
df_MCMC = CSV.read(path, df_MCMC)

In [ ]:
# Remove burn-in
deleteat!(df_MCMC,collect(1:500))

In [ ]:
# Plot poluation chains

# Plot density
l = @layout [a a; a a; a a; a a;]
plot_rho1bar_chain = plot(df_MCMC[:,"iteration"],df_MCMC[:,"rhobar[1]"],linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\mu_\rho")
xlabel!("Iteration")
ylabel!(L"\mu_\rho/\textrm{(\mu g/day)}^{-1}")
plot_rho1bar_density = plot(density(df_MCMC[:,"rhobar[1]"],legend=false),linewidth = 0.5, colour = 1, label = "")
# xlims!(0, 3)
title!("\nDensity for "*L"\mu_\rho")
xlabel!(L"\mu_\rho/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
plot_rho1sd_chain = plot(df_MCMC[:,"iteration"], df_MCMC[:,"rhotau[1]"],linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_\rho")
xlabel!("Iteration")
ylabel!(L"\sigma_\rho/\textrm{(\mu g/day)}^{-1}")
# ylims!(0,5)
plot_rho1sd_density = plot(density(df_MCMC[:,"rhotau[1]"],legend=false),linewidth = 0.5, colour = 1, label = "")
# xlims!(0, 0.2)
title!("Density for "*L"\sigma_\rho")
xlabel!(L"\sigma_\rho/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
plot_rho2bar_chain = plot(df_MCMC[:,"iteration"],df_MCMC[:,"rhobar[2]"],linewidth = 0.5, colour = 1, label = "")
title!("\nChain for "*L"\mu_\delta")
xlabel!("Iteration")
ylabel!(L"\mu_\delta/\textrm{(\mu g/day)}^{-1}")
plot_rho2bar_density = plot(density(df_MCMC[:,"rhobar[2]"],legend=false),linewidth = 0.5, colour = 1, label = "")
# xlims!(0, 0.2)
title!("\nDensity for "*L"\mu_\delta")
xlabel!(L"\mu_\delta/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
plot_rho2sd_chain = plot(df_MCMC[:,"iteration"], df_MCMC[:,"rhotau[2]"],linewidth = 0.5, colour = 1, label = "")
title!("Chain for "*L"\sigma_\delta")
xlabel!("Iteration")
ylabel!(L"\sigma_\delta/\textrm{(\mu g/day)}^{-1}")
plot_rho2sd_density = plot(density(df_MCMC[:,"rhotau[2]"],legend=false),linewidth = 0.5, colour = 1, label = "")
# xlims!(0, 0.2)
title!("Density for "*L"\sigma_\delta")
xlabel!(L"\sigma_\delta/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
figcombined = plot(plot_rho1bar_chain,plot_rho1bar_density,plot_rho1sd_chain,plot_rho1sd_density,plot_rho2bar_chain,
    plot_rho2bar_density,plot_rho2sd_chain,plot_rho2sd_density,layout = l,size = (700, 1000), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figcombined,figpath*"Popluation_MCMC_$(effect).png")
savefig(figcombined,figpath*"Popluation_MCMC_$(effect).pdf")
savefig(figcombined,figpath*"Popluation_MCMC_$(effect).svg")

In [ ]:
# Extract relevant data
pID = 5
df_p = df_D[df_D.patientID .== pID, :]

# Create copy of df_p with constant dose - 45/7
df_p_const = copy(df_p)
df_p_const.IFN[df_p_const.days.>=0] .= 45/7

# Extract latest data point
maxDays = maximum(df_p.days)

# Extract VAF
pVAF = df_p.JAK

# Choose treatment effect
effect = "py0dy1"

# Choose time for calculating VAF
pDays = 0:30:75*30.4
pDays2 = df_p.days[.!isnan.(df_p.JAK)]
pDays_const = [0,2*365]

# Load standard parameters
include("model_default_param.jl");

# Storage
VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
VAF_mat_const = Array{Float64, 2}(undef, length(pDays_const), length(df_MCMC[:,"iteration"]))
p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

# Calculate VAF in loop
for j in eachindex(df_MCMC[:,"iteration"])
    # Set rho values
    rho = [df_MCMC[j,"theta_$(pID)[1]"], df_MCMC[j,"theta_$(pID)[2]"]]

    # Set initial JAK
    initJAK = df_MCMC[j,"theta_$(pID)[3]"]

    # Calculate VAF using function
    VAF_mat[:,j], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
    VAF_mat_points[:,j], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
    VAF_mat_const[:,j], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)

    # Save all variables
    p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
    a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
    p_mat[:,j,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
end

# Calculate RMSE based on mean at the points
RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

# Calculate mean width of 95% CI
mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

# Calculate mean and median parameter values
mean_param_1 = mean(df_MCMC[:,"theta_$(pID)[1]"])
mean_param_2 = mean(df_MCMC[:,"theta_$(pID)[2]"])
median_param_1 = median(df_MCMC[:,"theta_$(pID)[1]"])
median_param_2 = median(df_MCMC[:,"theta_$(pID)[2]"])

# Plot results
figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
        linewidth = 3, label = "", colour= :blue1)
plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
      linewidth = 3, label = "", colour= :blue1)
title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
       *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
xlabel!(L"t"*"/months")
ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
xlims!(0, 75)
ylims!(0, 20)
p2 = twinx()
plot!(p2, pDays/30.4, quantile.(eachrow(VAF_mat), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat), 0.975)*100, fillalpha = 0.35, 
     colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
# plot!(p2, pDays[pDays.<=maxDays]/30.4, mean(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
#          label = L"JAK2"*" VAF - Mean", colour= :darkgoldenrod1)
# plot!(p2, pDays[pDays.>maxDays]/30.4, mean(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
#       label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
plot!(p2, pDays[pDays.<=maxDays]/30.4, median(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
      label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
plot!(p2, pDays[pDays.>maxDays]/30.4, median(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
      label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
scatter!(p2, df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
ylabel!(p2, L"JAK2"*" VAF/%")
xlims!(p2, 0, 75)
ylims!(p2, 0, 100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm)

In [ ]:
# Plot 4 patient plots in one figure

# Choose patients
p_vec = [2,3,4,5]

# Create empty plots
fig1 = plot()
fig2 = plot()
fig3 = plot()
fig4 = plot()

@showprogress dt=1 desc="Plotting and saving results for all patients..." for i=1:4
    # Extract relevant data
    pID = p_vec[i]
    df_p = df_D[df_D.patientID .== pID, :]

    # Create copy of df_p with constant dose - 45/7
    df_p_const = copy(df_p)
    df_p_const.IFN[df_p_const.days.>=0] .= 45/7

    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:30:75*30.4
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pDays_const = [0,2*365]

    # Load standard parameters
    include("model_default_param.jl");

    # Storage
    VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
    VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
    VAF_mat_const = Array{Float64, 2}(undef, length(pDays_const), length(df_MCMC[:,"iteration"]))
    p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

    # Calculate VAF in loop
    for j in eachindex(df_MCMC[:,"iteration"])
        # Set rho values
        rho = [df_MCMC[j,"theta_$(pID)[1]"], df_MCMC[j,"theta_$(pID)[2]"]]

        # Set initial JAK
        initJAK = df_MCMC[j,"theta_$(pID)[3]"]

        # Calculate VAF using function
        VAF_mat[:,j], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
        VAF_mat_points[:,j], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
        VAF_mat_const[:,j], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)

        # Save all variables
        p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
        p_mat[:,j,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
    end

    # Calculate RMSE based on mean at the points
    RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])

    # Calculate mean width of 95% CI
    mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))

    # Calculate mean and median parameter values
    mean_param_1 = mean(df_MCMC[:,"theta_$(pID)[1]"])
    mean_param_2 = mean(df_MCMC[:,"theta_$(pID)[2]"])
    median_param_1 = median(df_MCMC[:,"theta_$(pID)[1]"])
    median_param_2 = median(df_MCMC[:,"theta_$(pID)[2]"])

   # Plot results
    figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
    plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
    title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
           *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
    xlims!(0, 75)
    ylims!(0, 20)
    p2 = twinx()
    plot!(p2, pDays/30.4, quantile.(eachrow(VAF_mat), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat), 0.975)*100, fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#     plot!(p2, pDays[pDays.<=maxDays]/30.4, mean(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
#              label = L"JAK2"*" VAF - Mean", colour= :darkgoldenrod1)
#     plot!(p2, pDays[pDays.>maxDays]/30.4, mean(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
#           label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
    plot!(p2, pDays[pDays.<=maxDays]/30.4, median(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 6, 
          label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
    plot!(p2, pDays[pDays.>maxDays]/30.4, median(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 6, 
          label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
    scatter!(p2, df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
    ylabel!(p2, L"JAK2"*" VAF/%")
    xlims!(p2, 0, 75)
    ylims!(p2, 0, 100)
    plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    
    if i==1
        fig1 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==2
        fig2 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==3
        fig3 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==4
        fig4 = plot(figVAF, size = (700, 500), margin=5mm)
    end
end

# Plot all 4 figures
figCombined = plot(fig1, fig2, fig3, fig4, layout=grid(2,2), size = (1500, 1000), margin = 5mm, leftmargin = 8mm, 
                   rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figCombined,figpath*"Representative_patients_VAF_$(effect).png")
savefig(figCombined,figpath*"Representative_patients_VAF_$(effect).pdf")
savefig(figCombined,figpath*"Representative_patients_VAF_$(effect).svg")

In [ ]:
# Plot 4 patient plots in one figure - cells

# Choose patients
p_vec = [2,3,4,5]

# Create empty plots
fig1 = plot()
fig2 = plot()
fig3 = plot()
fig4 = plot()
fig5 = plot()

@showprogress dt=1 desc="Plotting and saving results for all patients..." for i=1:4
    # Extract relevant data
    pID = p_vec[i]
    df_p = df_D[df_D.patientID .== pID, :]

    # Create copy of df_p with constant dose - 45/7
    df_p_const = copy(df_p)
    df_p_const.IFN[df_p_const.days.>=0] .= 45/7

    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:7*365
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pDays_const = [0,2*365]

    # Load standard parameters
    include("model_default_param.jl");

    # Storage
    VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
    VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
    VAF_mat_const = Array{Float64, 2}(undef, length(pDays_const), length(df_MCMC[:,"iteration"]))
    p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

    # Calculate VAF in loop
    for j in eachindex(df_MCMC[:,"iteration"])
        # Set rho values
        rho = [df_MCMC[j,"theta_$(pID)[1]"], df_MCMC[j,"theta_$(pID)[2]"]]

        # Set initial JAK
        initJAK = df_MCMC[j,"theta_$(pID)[3]"]

        # Calculate VAF using function
        VAF_mat[:,j], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
        VAF_mat_points[:,j], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
        VAF_mat_const[:,j], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)

        # Save all variables
        p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
        p_mat[:,j,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
    end

    # Extract data points that are non-NAN and for time greater than 0
    pWBC = df_p.WBC[.!isnan.(df_p.WBC)]
    WBC_days = df_p.days[.!isnan.(df_p.WBC)]
    pWBC = pWBC[WBC_days.>=0]
    WBC_days = WBC_days[WBC_days.>=0]

    # Storage
    pWBC_model = Array{Float64, 1}(undef, length(WBC_days))

    # Extract values in loop
    for k=1:length(WBC_days)
        ID = findfirst(days -> days == WBC_days[k], pDays)
        pWBC_model[k] = median(p_mat[ID,:,3]+p_mat[ID,:,6])/(30.5*5)
    end

    # Setup dataframe
    data_WBC = DataFrame(x = pWBC_model, y=pWBC*10^9)

    # Setup model and calculate optimal fit
    model_WBC = lm(@formula(y ~ 0 + x), data_WBC)

    # Extract fit coefficient
    kWBC = coef(model_WBC)[1]
    
    # Calculate RMSE based on mean at the points
    RMSE_WBC = rmsd(kWBC*pWBC_model,pWBC*10^9)
    
    # Calculate CV
    CV_WBC = std(kWBC*pWBC_model-pWBC*10^9)/mean(kWBC*pWBC_model-pWBC*10^9)

    # Setup prediction interval
    data_WBC_new = DataFrame(x = pWBC_model)

    # Calculate prediction interval
    co_WBC = predict(model_WBC, data_WBC_new, interval = :confidence, level = 0.95)
    pr_WBC = predict(model_WBC, data_WBC_new, interval = :prediction, level = 0.95)

    # Convert to Float64
    co_WBC[!,:] = convert.(Float64,co_WBC)
    pr_WBC[!,:] = convert.(Float64,pr_WBC)
    
    # Extract data points that are non-NAN and for time greater than 0
    pTRC = df_p.TRC[.!isnan.(df_p.TRC)]
    TRC_days = df_p.days[.!isnan.(df_p.TRC)]
    pTRC = pTRC[TRC_days.>=0]
    TRC_days = TRC_days[TRC_days.>=0]

    # Storage
    pTRC_model = Array{Float64, 1}(undef, length(TRC_days))

    # Extract values in loop
    for k=1:length(TRC_days)
        ID = findfirst(days -> days == TRC_days[k], pDays)
        pTRC_model[k] = median(p_mat[ID,:,3]+p_mat[ID,:,6])/(30.5*5)
    end

    # Setup dataframe
    data_TRC = DataFrame(x = pTRC_model, y=pTRC*10^9)

    # Setup model and calculate optimal fit
    model_TRC = lm(@formula(y ~ 0 + x), data_TRC)

    # Extract fit coefficient
    kTRC = coef(model_TRC)[1]
    
    # Calculate RMSE based on mean at the points
    RMSE_TRC = rmsd(kTRC*pTRC_model,pTRC*10^9)
    
    # Calculate CV for errors
    CV_TRC = std(kWBC*pWBC_model-pWBC*10^9)/mean(kWBC*pWBC_model-pWBC*10^9)

    # Setup prediction interval
    data_TRC_new = DataFrame(x = pTRC_model)

    # Calculate prediction interval
    co_TRC = predict(model_TRC, data_TRC_new, interval = :confidence, level = 0.95)
    pr_TRC = predict(model_TRC, data_TRC_new, interval = :prediction, level = 0.95)

    # Convert to Float64
    co_TRC[!,:] = convert.(Float64,co_TRC)
    pr_TRC[!,:] = convert.(Float64,pr_TRC)

    # Plot results
    # Plot WBC
    figWBC = plot(pDays[pDays.<=maxDays]/30.4, kWBC*median(p_mat[pDays.<=maxDays,:,3]+p_mat[pDays.<=maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black)
    plot!(pDays[pDays.>maxDays]/30.4, kWBC*median(p_mat[pDays.>maxDays,:,3]+p_mat[pDays.>maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black, linestyle = :dash)
    plot!(WBC_days/30.4, co_WBC.lower, fillrange = co_WBC.upper, fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Prediction\ Interval}", linealpha = 0)
    plot!([0,75],[3.5*10^9,3.5*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    plot!([0,75],[8.8*10^9,8.8*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    scatter!(df_p.days./30.4, df_p.WBC*10^9, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.25)
    title!("Leukocytes for patient $(pID),\n" * "RMSE = $(round(RMSE_WBC; sigdigits=3)), "*L"k_\textrm{L}="
           *"$(round(kWBC; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("Cells/L")
    xlims!(-Inf, 75)
    ylims!(0,30*10^9)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
    # Plot TRC
    figTRC = plot(pDays[pDays.<=maxDays]/30.4, kTRC*median(p_mat[pDays.<=maxDays,:,3]+p_mat[pDays.<=maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
     colour= :black)
    plot!(pDays[pDays.>maxDays]/30.4, kTRC*median(p_mat[pDays.>maxDays,:,3]+p_mat[pDays.>maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black, linestyle = :dash)
    plot!(TRC_days/30.4, co_TRC.lower, fillrange = co_TRC.upper, fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Prediction\ Interval}", linealpha = 0)
    plot!([0,75],[145*10^9,145*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    plot!([0,75],[390*10^9,390*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    scatter!(df_p.days./30.4, df_p.TRC*10^9, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.25)
    title!("Thrombocytes for patient $(pID),\n" * "RMSE = $(round(RMSE_TRC; sigdigits=3)), "*L"k_\textrm{T}="
           *"$(round(kTRC; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("Cells/L")
    xlims!(-Inf, 75)
    ylims!(0,11*10^11)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
    l = @layout [a; a;]
    if i==1
        fig1 = plot(figWBC, figTRC, size = (700, 1000), margin=5mm)
    elseif i==2
        fig2 = plot(figWBC, figTRC, size = (700, 1000), margin=5mm)
    elseif i==3
        fig3 = plot(figWBC, figTRC, size = (700, 1000), margin=5mm)
    elseif i==4
        fig4 = plot(figWBC, figTRC, size = (700, 1000), margin=5mm)
    end
end

# Plot all 5 figures
figCombined = plot(fig1, fig2, fig3, fig4, layout=grid(2,2), size = (2500, 1500), margin = 10mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figCombined,figpath*"Representative_patients_Cells_$(effect).png")
savefig(figCombined,figpath*"Representative_patients_Cells_$(effect).pdf")
savefig(figCombined,figpath*"Representative_patients_Cells_$(effect).svg")

In [ ]:
# Define doses we want to investigate we want to investigate
cI_lin = range(0,180,51)./7

# Number of patients
P = length(unique_patients)

# Storage
real_VAF = Float64[]
predicted_VAF = Float64[]
R_factor = fill(NaN, P, length(df_MCMC[:,"iteration"]))
R_factor_mat = zeros(length(cI_lin),length(df_MCMC[:,"iteration"]))
med_RMSE_vec = Array{Float64, 1}(undef, P)
med_RMSE_WBC_vec = Array{Float64, 1}(undef, P)
med_RMSE_TRC_vec = Array{Float64, 1}(undef, P)
med_Rhat_vec = Array{Float64, 1}(undef, P)
n_data_vec = Array{Float64, 1}(undef, P) 
mean_rho_py0_vec = Array{Float64, 1}(undef, P)
mean_rho_dy1_vec = Array{Float64, 1}(undef, P)
median_rho_py0_vec = Array{Float64, 1}(undef, P)
median_rho_dy1_vec = Array{Float64, 1}(undef, P)
k_WBC_vec = Array{Float64, 1}(undef, P)
k_TRC_vec = Array{Float64, 1}(undef, P)

# Setup for stamp plots
stamp_plot_1 = Any[]
stamp_plot_2 = Any[]
stamp_plot_3 = Any[]
stamp_plot_4 = Any[]


# Plot for all patients in loop
@showprogress dt=1 desc="Plotting and saving results for all patients..." for i=1:P
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Create copy of df_p with constant dose - 45/7
    df_p_const = copy(df_p)
    df_p_const.IFN[df_p_const.days.>=0] .= 45/7
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:7*365
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pDays_const = [0,2*365]
    pDays_plot = pDays[1:30:end]

    # Load standard parameters
    include("model_default_param.jl");

    # Storage
    VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
    VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
    VAF_mat_const = Array{Float64, 2}(undef, length(pDays_const), length(df_MCMC[:,"iteration"]))
    p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

    # Calculate VAF in loop
    for j in eachindex(df_MCMC[:,"iteration"])
        # Set rho values
        rho = [df_MCMC[j,"theta_$(pID)[1]"], df_MCMC[j,"theta_$(pID)[2]"]]

        # Set initial JAK
        initJAK = df_MCMC[j,"theta_$(pID)[3]"]

        # Calculate VAF using function
        VAF_mat[:,j], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
        VAF_mat_points[:,j], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
        VAF_mat_const[:,j], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)
        
        # Save all variables
        p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
        a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
        p_mat[:,j,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
        
    end
    VAF_mat_plot = VAF_mat[1:30:end,:]
    
    # Calculate RMSE based on mean at the points
    RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])
    
    # Save mean RMSE
    med_RMSE_vec[i] = RMSE
    
    # Calculate Rhat based on mean at the points
    sumE = sum((median(VAF_mat_points, dims=2).-df_p.JAK[.!isnan.(df_p.JAK)]).^2)
    sumJ = sum((df_p.JAK[.!isnan.(df_p.JAK)].-mean(df_p.JAK[.!isnan.(df_p.JAK)])).^2)
    n_data_vec[i] = length(df_p.JAK[.!isnan.(df_p.JAK)])
    n_param = 3
    med_Rhat_vec[i] = 1-sumE/sumJ*(n_data_vec[i]-1)/(n_data_vec[i]-n_param-1)
        
    # Save real and predicted VAF
    append!(real_VAF, df_p.JAK[.!isnan.(df_p.JAK)])
    append!(predicted_VAF, median(VAF_mat_points, dims=2))
    
    # Calculate mean and median parameter values
    mean_param_1 = mean(df_MCMC[:,"theta_$(pID)[1]"])
    mean_param_2 = mean(df_MCMC[:,"theta_$(pID)[2]"])
    median_param_1 = median(df_MCMC[:,"theta_$(pID)[1]"])
    median_param_2 = median(df_MCMC[:,"theta_$(pID)[2]"])
    
    # Save mean and median parameter values
    mean_rho_py0_vec[i] = mean_param_1
    mean_rho_dy1_vec[i] = mean_param_2
    median_rho_py0_vec[i] = median_param_1
    median_rho_dy1_vec[i] = median_param_2

    # Calculate R factor
    R_factor[i,:] = 100*(1 .- VAF_mat_const[2,:]./VAF_mat_const[1,:])
    
    # Extract data points that are non-NAN and for time greater than 0
    pWBC = df_p.WBC[.!isnan.(df_p.WBC)]
    WBC_days = df_p.days[.!isnan.(df_p.WBC)]
    pWBC = pWBC[WBC_days.>=0]
    WBC_days = WBC_days[WBC_days.>=0]

    # Storage
    pWBC_model = Array{Float64, 1}(undef, length(WBC_days))

    # Extract values in loop
    for k=1:length(WBC_days)
        ID = findfirst(days -> days == WBC_days[k], pDays)
        pWBC_model[k] = median(p_mat[ID,:,3]+p_mat[ID,:,6])/(30.5*5)
    end
    

    # Setup dataframe
    data_WBC = DataFrame(x = pWBC_model, y=pWBC*10^9)

    # Setup model and calculate optimal fit
    model_WBC = lm(@formula(y ~ 0 + x), data_WBC)

    # Extract fit coefficient
    kWBC = coef(model_WBC)[1]
    
    # Calculate RMSE based on mean at the points
    RMSE_WBC = rmsd(kWBC*pWBC_model,pWBC*10^9)/((3.5*10^9+8.8*10^9)/2)
    
    # Save values
    k_WBC_vec[i] = kWBC
    med_RMSE_WBC_vec[i] = RMSE_WBC

    # Setup prediction interval
    data_WBC_new = DataFrame(x = pWBC_model)

    # Calculate prediction interval
    co_WBC = predict(model_WBC, data_WBC_new, interval = :confidence, level = 0.95)
    pr_WBC = predict(model_WBC, data_WBC_new, interval = :prediction, level = 0.95)

    # Convert to Float64
    co_WBC[!,:] = convert.(Float64,co_WBC)
    pr_WBC[!,:] = convert.(Float64,pr_WBC)
    
    # Extract data points that are non-NAN and for time greater than 0
    pTRC = df_p.TRC[.!isnan.(df_p.TRC)]
    TRC_days = df_p.days[.!isnan.(df_p.TRC)]
    pTRC = pTRC[TRC_days.>=0]
    TRC_days = TRC_days[TRC_days.>=0]

    # Storage
    pTRC_model = Array{Float64, 1}(undef, length(TRC_days))

    # Extract values in loop
    for k=1:length(TRC_days)
        ID = findfirst(days -> days == TRC_days[k], pDays)
        pTRC_model[k] = median(p_mat[ID,:,3]+p_mat[ID,:,6])/(30.5*5)
    end

    # Setup dataframe
    data_TRC = DataFrame(x = pTRC_model, y=pTRC*10^9)

    # Setup model and calculate optimal fit
    model_TRC = lm(@formula(y ~ 0 + x), data_TRC)

    # Extract fit coefficient
    kTRC = coef(model_TRC)[1]
    
    # Calculate RMSE based on mean at the points
    RMSE_TRC = rmsd(kTRC*pTRC_model,pTRC*10^9)/((145*10^9+390*10^9)/2)
    
    # Save values
    k_TRC_vec[i] = kTRC
    med_RMSE_TRC_vec[i] = RMSE_TRC

    # Setup prediction interval
    data_TRC_new = DataFrame(x = pTRC_model)

    # Calculate prediction interval
    co_TRC = predict(model_TRC, data_TRC_new, interval = :confidence, level = 0.95)
    pr_TRC = predict(model_TRC, data_TRC_new, interval = :prediction, level = 0.95)

    # Convert to Float64
    co_TRC[!,:] = convert.(Float64,co_TRC)
    pr_TRC[!,:] = convert.(Float64,pr_TRC)
    
    for k=1:length(cI_lin)
        # Create copy of df_p with constant dose 
        df_p_const = copy(df_p)
        df_p_const.IFN[df_p_const.days.>=0] .= cI_lin[i]


        # Extract VAF
        pVAF = df_p.JAK

        # Choose treatment effect
        effect = "py0dy1"

        # Choose time for calculating VAF
        pDays_const = [0,2*365]

        # Load standard parameters
        include("model_default_param.jl")

        # Storage
        VAF_mat_const = Array{Float64, 2}(undef, length(pDays_const), length(df_MCMC[:,"iteration"]))

        # Calculate VAF in loop
        for j in eachindex(df_MCMC[:,"iteration"])
            # Set rho values
            rho = [df_MCMC[j,"theta_$(pID)[1]"], df_MCMC[j,"theta_$(pID)[2]"]]

            # Set initial JAK
            initJAK = df_MCMC[j,"theta_$(pID)[3]"]

            # Calculate VAF using function
            VAF_mat_const[:,j], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)
        end

        # Calculate R-factor and append
        R_factor_mat[k,:] = 100*(1 .- VAF_mat_const[end,:]./df_MCMC[:,"theta_$(pID)[3]"])
    end

    # Plot results
    figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
    plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
    title!(L"JAK2"*" VAF for patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
           *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
    xlims!(0, 75)
    ylims!(0, 20)
    p2 = twinx()
    plot!(p2, pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
          fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#     plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, mean(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
#              label = L"JAK2"*" VAF - Mean", colour= :darkgoldenrod1)
#     plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, mean(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
#           label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
    plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
          label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
    plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
          label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
    scatter!(p2, df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
    ylabel!(p2, L"JAK2"*" VAF/%")
    xlims!(p2, 0, 75)
    ylims!(p2, 0, 100)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    figVAF = plot(figVAF, size = (700, 500), margin=5mm, leftmargin = 8mm)
    
    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figVAF,figpath*"Hierarchical_$(effect)_p$(pID).png")
    savefig(figVAF,figpath*"Hierarchical_$(effect)_p$(pID).pdf")
    savefig(figVAF,figpath*"Hierarchical_$(effect)_p$(pID).svg")
    
    # Stamp plot
    figVAF_stamp = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
    plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
    title!(L"JAK2"*" VAF for Patient $(pID)")
    xlabel!(L"t"*"/months")
    ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
    xlims!(0, 75)
    ylims!(0, 20)
    p2 = twinx()
    plot!(p2, pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100,
          fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#     plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, mean(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
#              label = L"JAK2"*" VAF - Mean", colour= :darkgoldenrod1)
#     plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, mean(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
#           label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
    plot!(p2, pDays[pDays.<=maxDays]/30.4, median(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
          label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
    plot!(p2, pDays[pDays.>maxDays]/30.4, median(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
          label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
    scatter!(p2, df_p.days./30.4, df_p.JAK*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
    ylabel!(p2, L"JAK2"*" VAF/%")
    xlims!(p2, 0, 75)
    ylims!(p2, 0, 100)
    figVAF_stamp = plot(figVAF_stamp, size = (700, 500), margin=5mm, leftmargin = 8mm)
    # plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10)

    if i<=16
        push!(stamp_plot_1,figVAF_stamp)
    elseif i<=32
        push!(stamp_plot_2,figVAF_stamp)
    elseif i<=48
        push!(stamp_plot_3,figVAF_stamp)
    else
        push!(stamp_plot_4,figVAF_stamp)
    end
    
    # Plot chains
    l = @layout [a a; a a; a a; a a;]
    plot_rho1_chain = plot(df_MCMC.iteration,df_MCMC[:,"theta_$(pID)[1]"],linewidth = 0.5, colour = 1, label = "")
    title!("\nChain for "*L"\rho")
    xlabel!("Iteration")
    ylabel!(L"\rho/\mathrm{(\frac{\mu g}{day}})^{-1}")
    plot_rho1_density = plot(density(df_MCMC[:,"theta_$(pID)[1]"],legend=false),linewidth = 0.5, colour = 1, label = "")
    xlims!(0,1.1)
    title!("\nDensity for "*L"\rho")
    xlabel!(L"\rho/\mathrm{(\frac{\mu g}{day}})^{-1}")
    ylabel!("pdf/1")
    plot_rho2_chain = plot(df_MCMC.iteration,df_MCMC[:,"theta_$(pID)[2]"],linewidth = 0.5, colour = 1, label = "")
    title!("Chain for "*L"\delta")
    xlabel!("Iteration")
    ylabel!(L"\delta/\mathrm{(\frac{\mu g}{day}})^{-1}")
    plot_rho2_density = plot(density(df_MCMC[:,"theta_$(pID)[2]"],legend=false),linewidth = 0.5, colour = 1, label = "")
    title!("Density for "*L"\delta")
    xlims!(0,1.1)
    xlabel!(L"\delta/\mathrm{(\frac{\mu g}{day}})^{-1}")
    ylabel!("pdf/1")
    plot_initJAK_chain = plot(df_MCMC.iteration,df_MCMC[:,"theta_$(pID)[3]"]*100,linewidth = 0.5, colour = 1, label = "")
    title!(L"Chain for $v_0$")
    xlabel!("Iteration")
    ylabel!(L"$v_0$/%")
    plot_initJAK_density = plot(density(df_MCMC[:,"theta_$(pID)[3]"]*100,legend=false),linewidth = 0.5, colour = 1, label = "")
    xlims!(0,100)
    title!(L"Density for $v_0$")
    xlabel!(L"$v_0$/%")
    ylabel!("pdf/1")
    plot_sigma_chain = plot(df_MCMC.iteration,df_MCMC[:,"theta_$(pID)[4]"]*100,linewidth = 0.5, colour = 1, label = "")
    title!("Chain for "*L"\sigma")
    xlabel!("Iteration")
    ylabel!(L"\sigma/\%")
    plot_sigma_density = plot(density(df_MCMC[:,"theta_$(pID)[4]"]*100,legend=false),linewidth = 0.5, colour = 1, label = "")
    title!("Density for "*L"\sigma")
    xlims!(0,100)
    xlabel!(L"\sigma/\%")
    ylabel!("pdf/1")
    figMCMC = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_initJAK_chain,plot_initJAK_density,
        plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000), margin=5mm)
    
    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figMCMC,figpath*"Hierarchical_Chain_$(effect)_p$(pID).png")
    savefig(figMCMC,figpath*"Hierarchical_Chain_$(effect)_p$(pID).pdf")
    savefig(figMCMC,figpath*"Hierarchical_Chain_$(effect)_p$(pID).svg")

    # Make correlation plot
    figCorr = corrplot([df_MCMC[:,"theta_$(pID)[1]"] df_MCMC[:,"theta_$(pID)[2]"] df_MCMC[:,"theta_$(pID)[3]"] df_MCMC[:,"theta_$(pID)[4]"]],
         labels = [L"\rho", L"\delta", 
                   L"v_0", L"\sigma"], title = "Correlation plot for patient $(pID)", title_location = :center)
    
    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_p$(pID).png")
    savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_p$(pID).pdf")
    savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_p$(pID).svg")
    
    # Plot WBC
    figWBC = plot(WBC_days/30.4, co_WBC.lower, fillrange = co_WBC.upper, fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Prediction\ Interval}", linealpha = 0)
    plot!(pDays[pDays.>maxDays]/30.4, kWBC*median(p_mat[pDays.>maxDays,:,3]+p_mat[pDays.>maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black, linestyle = :dash)
    plot!(pDays[pDays.<=maxDays]/30.4, kWBC*median(p_mat[pDays.<=maxDays,:,3]+p_mat[pDays.<=maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black)
    plot!([0,75],[3.5*10^9,3.5*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    plot!([0,75],[8.8*10^9,8.8*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    scatter!(df_p.days./30.4, df_p.WBC*10^9, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
    title!("Leukocytes for patient $(pID),\n" * L"\textrm{RMSE}_\textrm{norm}" * "= $(round(RMSE_WBC; sigdigits=3)), "*L"k_\textrm{L}="
           *"$(round(kWBC; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("Cells/L")
    xlims!(-Inf, 75)
    ylims!(0,30*10^9)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
    # Plot TRC
    figTRC = plot(TRC_days/30.4, co_TRC.lower, fillrange = co_TRC.upper, fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Prediction\ Interval}", linealpha = 0)
    plot!(pDays[pDays.>maxDays]/30.4, kTRC*median(p_mat[pDays.>maxDays,:,3]+p_mat[pDays.>maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
         colour= :black, linestyle = :dash)
    plot!(pDays[pDays.<=maxDays]/30.4, kTRC*median(p_mat[pDays.<=maxDays,:,3]+p_mat[pDays.<=maxDays,:,6],dims=2)/(30.5*5), linewidth = 3, label = "",
     colour= :black)
    plot!([0,75],[145*10^9,145*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    plot!([0,75],[390*10^9,390*10^9], linewidth = 3, label = "", colour= :green1, linestyle = :dash)
    scatter!(df_p.days./30.4, df_p.TRC*10^9, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
    title!("Thrombocytes for patient $(pID),\n" * L"\textrm{RMSE}_\textrm{norm}" * "= $(round(RMSE_TRC; sigdigits=3)), "*L"k_\textrm{T}="
           *"$(round(kTRC; sigdigits=3))")
    xlabel!(L"t"*"/months")
    ylabel!("Cells/L")
    xlims!(-Inf, 75)
    ylims!(0,11*10^11)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
    
    
    l = @layout [a; a;]
    figP = plot(figWBC, figTRC, size = (1200, 1000), margin=5mm)
    
    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figP,figpath*"Hierarchical_cell_$(effect)_p$(pID).png")
    savefig(figP,figpath*"Hierarchical_cell_$(effect)_p$(pID).pdf")
    savefig(figP,figpath*"Hierarchical_cell_$(effect)_p$(pID).svg")
    
    # Plot dose-response
    figR = plot(cI_lin*7,quantile.(eachrow(R_factor_mat),0.025), fillrange = quantile.(eachrow(R_factor_mat),0.975), 
            fillalpha = 0.35, colour = :grey60, label = "", linealpha = 0)
    plot!(cI_lin*7,median(R_factor_mat,dims=2), linewidth = 3, label = "", colour= :blue1,
                xticks = (0:45:180, 0:45:180))
    title!(L"$R$-Factor as a function of IFN-$\alpha$" *"\n dose for patient $(pID)")
    xlabel!(L"IFN-$\alpha$/$\mathrm{\frac{\mu g}{week}}$")
    ylabel!(L"$R$-Factor/%")
    xlims!(0, 180)
    ylims!(-30, 100)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, margin = 5mm)
    
    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figR,figpath*"Hierarchical_dose_response_$(effect)_p$(pID).png")
    savefig(figR,figpath*"Hierarchical_dose_response_$(effect)_p$(pID).pdf")
    savefig(figR,figpath*"Hierarchical_dose_response_$(effect)_p$(pID).svg")
    
    # Plot full model output
    figS = plot(pDays/30.4, quantile.(eachrow(p_mat[:,:,1]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,1]), 0.975), fillalpha = 0.35, 
         colour = :green1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,1], dims=2), linewidth = 3, label=L"x_0",colour=:green1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,4]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,4]), 0.975), fillalpha = 0.35, 
         colour = :red1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,4], dims=2), linewidth = 3, label=L"y_0",colour=:red1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,1]+p_mat[:,:,4]), 0.025), 
          fillrange = quantile.(eachrow(p_mat[:,:,1]+p_mat[:,:,4]), 0.975), fillalpha = 0.35, 
          colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,1]+p_mat[:,:,4], dims=2), linewidth = 3, label=L"x_0+y_0",colour=:black,linestyle=:dash)
    title!("Stem Cells")
    xlabel!(L"t"*"/months")
    ylabel!("Cells"*L"/1")
    xlims!(0, 75)
    ylims!(0,2*10^5)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, legend=:none)
    figP =  plot(pDays/30.4, quantile.(eachrow(p_mat[:,:,2]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,2]), 0.975), fillalpha = 0.35, 
         colour = :green1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,2], dims=2), linewidth = 3, label=L"x_0",colour=:green1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,5]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,5]), 0.975), fillalpha = 0.35, 
         colour = :red1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,5], dims=2), linewidth = 3, label=L"y_0",colour=:red1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,2]+p_mat[:,:,5]), 0.025), 
          fillrange = quantile.(eachrow(p_mat[:,:,2]+p_mat[:,:,5]), 0.975), fillalpha = 0.35, 
          colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,2]+p_mat[:,:,5], dims=2), linewidth = 3, label=L"x_0+y_0",colour=:black,linestyle=:dash)
    title!("Progenitor and\n Precursor Cells")
    xlabel!(L"t"*"/months")
    ylabel!("Cells"*L"/1")
    xlims!(0, 75)
    ylims!(0,10*10^6)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, legend=:none)
    figM = plot(pDays/30.4, quantile.(eachrow(p_mat[:,:,3]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,3]), 0.975), fillalpha = 0.35, 
         colour = :green1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,3], dims=2), linewidth = 3, label=L"x_0",colour=:green1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,6]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,6]), 0.975), fillalpha = 0.35, 
         colour = :red1, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,6], dims=2), linewidth = 3, label=L"y_0",colour=:red1)
    plot!(pDays/30.4, quantile.(eachrow(p_mat[:,:,3]+p_mat[:,:,6]), 0.025), 
          fillrange = quantile.(eachrow(p_mat[:,:,3]+p_mat[:,:,6]), 0.975), fillalpha = 0.35, 
          colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,3]+p_mat[:,:,6], dims=2), linewidth = 3, label=L"x_0+y_0",colour=:black,linestyle=:dash)
    title!("Mature Cells")
    xlabel!(L"t"*"/months")
    ylabel!("Cells"*L"/1")
    xlims!(0, 75)
    ylims!(0,50*10^11)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, legend=:none)
    figa = plot(pDays/30.4, quantile.(eachrow(p_mat[:,:,7]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,7]), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,7], dims=2), linewidth = 3, label=L"a",colour=:darkgoldenrod1)
    title!("Cellular debris")
    xlabel!(L"t"*"/months")
    ylabel!(L"$a$/1")
    xlims!(0, 75)
    ylims!(0,2000)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, legend=:none)
    figs = plot(pDays/30.4, quantile.(eachrow(p_mat[:,:,8]), 0.025), fillrange = quantile.(eachrow(p_mat[:,:,8]), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
    plot!(pDays/30.4, median(p_mat[:,:,8], dims=2), linewidth = 3, label=L"a",colour=:darkgoldenrod1)
    title!("Cytokine signal")
    xlabel!(L"t"*"/months")
    ylabel!(L"$s$/1")
    xlims!(0, 75)
    ylims!(0,2)
    plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, legend=:none)
    figcombined = plot(figS, figP, figM, figa, figs, figVAF, figWBC, figTRC, figR, layout=grid(3, 3), link =:x, size = (2000, 2000), margin=5mm,
    rightmargin = 8mm, leftmargin = 8mm)

    # Save figure
    figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
    savefig(figcombined,figpath*"Hierarchical_full_$(effect)_p$(pID).png")
    savefig(figcombined,figpath*"Hierarchical_full_$(effect)_p$(pID).pdf")
    savefig(figcombined,figpath*"Hierarchical_full_$(effect)_p$(pID).svg")
end

In [ ]:
df_param

In [ ]:
# Calculate various quantities of interest
mean(med_RMSE_vec)

In [ ]:
sum(med_RMSE_vec.<=0.02)

In [ ]:
sum(med_RMSE_vec.<=0.02)/P

In [ ]:
sum(med_RMSE_vec.<=0.04)

In [ ]:
sum(med_RMSE_vec.<=0.04)/P

In [ ]:
sum(med_RMSE_vec.>0.1)

In [ ]:
sum(med_RMSE_vec.>0.1)/P

In [ ]:
unique_patients[med_RMSE_vec.>0.1]

In [ ]:
# Histogram of RMSE-values
fig_hist_RMSE = histogram(med_RMSE_vec, bins=range(0,0.2,21), label = "", colour = :darkgoldenrod1)
title!("Histogram of RMSE-values")
xlabel!("RMSE/1")
ylabel!("Number of Patients")
xlims!(0, 0.2)
ylims!(0, 10)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_hist_RMSE,figpath*"Histogram_RMSE_$(effect).png")
savefig(fig_hist_RMSE,figpath*"Histogram_RMSE_$(effect).pdf")
savefig(fig_hist_RMSE,figpath*"Histogram_RMSE_$(effect).svg")

In [ ]:
# Number of patients
P = length(unique_patients) 

# Storage
mean_param = Array{Float64, 2}(undef, P, 2)
med_param = Array{Float64, 2}(undef, P, 2)
med_initJAK = Array{Float64, 1}(undef, P)

# Extract mean parameter values
for i=1:P
    # Set pID
    pID = unique_patients[i]
    
    # Extract data
    mean_param[i,1] = mean(df_MCMC[:,"theta_$(pID)[1]"])
    mean_param[i,2] = mean(df_MCMC[:,"theta_$(pID)[2]"])
    med_param[i,1] = median(df_MCMC[:,"theta_$(pID)[1]"])
    med_param[i,2] = median(df_MCMC[:,"theta_$(pID)[2]"])
    med_initJAK[i] = median(df_MCMC[:,"theta_$(pID)[3]"])
end

# Calculate correlation
temp_corr = cor(med_param)
   
# Plot overall correlations
figCorr = corrplot(med_param, labels = [L"\rho", L"\delta"], 
                   title = "Correlation plot for mean\n parameter values, "*L"\rho="*"$(temp_corr[1,2])", title_location = :center)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_mean.png")
savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_mean.pdf")
savefig(figCorr,figpath*"Hierarchical_Correlation_$(effect)_mean.svg")

In [ ]:
# Storage
corr_vec = zeros(length(df_MCMC[:,"iteration"]))

# Extract rho and delta
rho_mat = Matrix(df_MCMC[:,7:4:end-1])
delta_mat = Matrix(df_MCMC[:,8:4:end-1])

# Calculate correlation for all samples in loop
for i in eachindex(df_MCMC[:,"iteration"])
    # Extract samples
    rho_vec = rho_mat[i,:]
    delta_vec = delta_mat[i,:]
    
    # Calculate correlation
    corr_vec[i] = cor(rho_vec, delta_vec)
end

In [ ]:
[quantile(corr_vec,0.025), mean(corr_vec), quantile(corr_vec,0.975)]

In [ ]:
# Storage
corr_vec = zeros(size(df_MCMC[:,"theta_$(2)[1]"]))

# Extract variables
df_rho_p_y_0 = df_MCMC[:,7:4:end-1]
df_rho_p_y_0 = Matrix(df_rho_p_y_0)
df_rho_d_y_1 = df_MCMC[:,8:4:end-1]
df_rho_d_y_1 = Matrix(df_rho_d_y_1)

# Calculate correlation in loop
for i=1:length(corr_vec)
    corr_vec[i] = cor(df_rho_p_y_0[i,:],df_rho_d_y_1[i,:])
end

# Output results
[quantile(corr_vec, 0.025), mean(corr_vec), quantile(corr_vec, 0.975)]

## Population distributions

In [ ]:
# Extract medians for plot
med_bar_1 = median(df_MCMC[:,"rhobar[1]"])
med_bar_2 = median(df_MCMC[:,"rhobar[2]"])
med_tau_1 = median(df_MCMC[:,"rhotau[1]"])
med_tau_2 = median(df_MCMC[:,"rhotau[2]"])

In [ ]:
# Define functions for distributions of py0tilde and dy1tilde
function f_py0(rho,c) 
    py0tilde = @. (2/(1+exp(rho*c)))*p.py0
end
function f_dy1(delta,c) 
    py0tilde = @. (1+delta*c)*p.dy1
end
function f_inv_py0(py0tilde,c)
    rho = @. 1/c*log(2*p.py0/py0tilde-1)
end
function f_inv_dy1(dy1tilde,c)
    delta = @. -(-dy1tilde + p.dy1)/(c*p.dy1)
end
function f_inv_py0_diff(py0tilde,c)
    fm = @. -2*p.py0/((2*p.py0-py0tilde)*py0tilde*c)
end
function f_inv_dy1_diff(dy1tilde,c)
    fm = @. 1/(c*p.dy1)
end

In [ ]:
# Calculate median parameter values 
med_py0 = @. (2/(1+exp(med_param[:,1]*45/7)))*p.py0
med_dy1 = @. (1+med_param[:,2]*45/7)*p.dy1

# Define vectors for plotting
rho_lin = range(0,1,101)
delta_lin = range(0,1,101)
py0tilde_lin = range(0,p.py0,101)
dy1tilde_lin = range(p.dy1,0.025,101)


# Plot distributions of estimated parameters given standard dose
# Plot population distributions - with histograms
l = @layout [a; a;]
plot_effect1 = histogram(med_py0, bins=range(0,1,21), label = "", normalize = :pdf)
# plot!(truncated(Normal(med_bar_1, med_tau_1), lower=0), linewidth = 3, colour = :red, label = "")
plot!(py0tilde_lin, pdf(truncated(Normal(med_bar_1, med_tau_1), lower=0),
                        f_inv_py0(py0tilde_lin,45/7)).*abs.(f_inv_py0_diff(py0tilde_lin,45/7)), 
            linewidth = 3, colour = :red, label = "")
plot!([p.py0,p.py0],[0,1000], linewidth = 3, colour = :green1, label = "", linestyle = :dash)
xlims!(0.0,1)
# ylims!(0.0,2.5)
title!("\nPopulation Distribution for "*L"\widetilde{p}_{y_0}"*"\n" *L"\mu_{\rho_{p_{y_0}}} = "*
        "$(round(med_bar_1; sigdigits=3)), "*
        L"\sigma_{\rho_{p_{y_0}}} = "*"$(round(med_tau_1; sigdigits=3))")
xlabel!(L"\widetilde{p}_{y_0}/1")
ylabel!("pdf/1")
plot_effect2 = histogram(med_dy1, label = "", normalize = :pdf)
# plot!(truncated(Normal(med_bar_2, med_tau_2), lower=0), 
#                     linewidth = 3, colour = :red, label = "")
plot!(dy1tilde_lin, pdf(truncated(Normal(med_bar_2, med_tau_2), lower=0),
                        f_inv_dy1(dy1tilde_lin,45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin,45/7)), 
            linewidth = 3, colour = :red, label = "")
plot!([p.dy1,p.dy1],[0,1000], linewidth = 3, colour = :green1, label = "", linestyle = :dash)
title!("\nPopulation Distribution for "*L"\widetilde{d}_{y_1}"*"\n" *L"\mu_{\rho_{d_{y_1}}} = "*
        "$(round(med_bar_2; sigdigits=3)), "*
        L"\sigma_{\rho_{d_{y_1}}} = "*"$(round(med_tau_2; sigdigits=3))")
# xlims!(0.0,1.0)
# ylims!(0.0,140)
xlabel!(L"\widetilde{d}_{y_1}/\textrm{day}^{-1}")
ylabel!("pdf/1")
figcombined = plot(plot_effect1,plot_effect2,layout = l,size = (700, 1000), margin=5mm)

# Save figure
# figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
# savefig(figcombined,figpath*"Popluation_distribution_bio_param_$(effect).png")
# savefig(figcombined,figpath*"Popluation_distribution_bio_param_$(effect).pdf")
# savefig(figcombined,figpath*"Popluation_distribution_bio_param_$(effect).svg")

In [ ]:
# Population distributions with CIs
# Define vectors for discretisation
rho_lin = range(0,1,1001)
delta_lin = range(0,1,1001)
py0tilde_lin = range(0.0001,p.py0,1001)
dy1tilde_lin = range(p.dy1,0.035,1001)
N = length(rho_lin)

# Storage
pdf_rho_mat = zeros(N,length(df_MCMC[:,"iteration"]))
pdf_delta_mat = zeros(N,length(df_MCMC[:,"iteration"]))
pdf_py0_mat = zeros(N,length(df_MCMC[:,"iteration"]))
pdf_dy1_mat = zeros(N,length(df_MCMC[:,"iteration"]))

# Calculate pdf values in loop
@showprogress dt=1 desc="Calculating population distribution for all samples..." for i=1:N
    for j=1:length(df_MCMC[:,"iteration"])
        pdf_rho_mat[i,j] = pdf(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0),rho_lin[i])
        pdf_delta_mat[i,j] = pdf(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0),rho_lin[i])
        pdf_py0_mat[i,j] = pdf(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0),
                               f_inv_py0(py0tilde_lin[i],45/7)).*abs.(f_inv_py0_diff(py0tilde_lin[i],45/7))
        pdf_dy1_mat[i,j] = pdf(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0),
                               f_inv_dy1(dy1tilde_lin[i],45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin[i],45/7))
    end
end

In [ ]:
# Storage
mean_rho_vec = zeros(length(df_MCMC[:,"iteration"]))
mean_delta_vec = zeros(length(df_MCMC[:,"iteration"]))
med_rho_vec = zeros(length(df_MCMC[:,"iteration"]))
med_delta_vec = zeros(length(df_MCMC[:,"iteration"]))
mode_rho_vec = zeros(length(df_MCMC[:,"iteration"]))
mode_delta_vec = zeros(length(df_MCMC[:,"iteration"]))
std_rho_vec = zeros(length(df_MCMC[:,"iteration"]))
std_delta_vec = zeros(length(df_MCMC[:,"iteration"]))
iqr_rho_vec = zeros(length(df_MCMC[:,"iteration"]))
iqr_delta_vec = zeros(length(df_MCMC[:,"iteration"]))

# Calculate summary statistics in loop
for j=1:length(df_MCMC[:,"iteration"])
    mean_rho_vec[j] = mean(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0))
    mean_delta_vec[j] = mean(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0))
    med_rho_vec[j] = median(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0))
    med_delta_vec[j] = median(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0))
    mode_rho_vec[j] = mode(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0))
    mode_delta_vec[j] = mode(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0))
    std_rho_vec[j] = std(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0))
    std_delta_vec[j] = std(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0))
    iqr_rho_vec[j] = iqr(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0))
    iqr_delta_vec[j] = iqr(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0))
end

In [ ]:
[[quantile(mean_rho_vec, 0.025), median(mean_rho_vec), quantile(mean_rho_vec, 0.975)],
 [quantile(mean_delta_vec, 0.025), median(mean_delta_vec), quantile(mean_delta_vec, 0.975)],
 [quantile(med_rho_vec, 0.025), median(med_rho_vec), quantile(med_rho_vec, 0.975)],
 [quantile(med_delta_vec, 0.025), median(med_delta_vec), quantile(med_delta_vec, 0.975)],
 [quantile(mode_rho_vec, 0.025), median(mode_rho_vec), quantile(mode_rho_vec, 0.975)],
 [quantile(mode_delta_vec, 0.025), median(mode_delta_vec), quantile(mode_delta_vec, 0.975)],
 [quantile(std_rho_vec, 0.025), median(std_rho_vec), quantile(std_rho_vec, 0.975)],
 [quantile(std_delta_vec, 0.025), median(std_delta_vec), quantile(std_delta_vec, 0.975)],
 [quantile(iqr_rho_vec, 0.025), median(iqr_rho_vec), quantile(iqr_rho_vec, 0.975)],
 [quantile(iqr_delta_vec, 0.025), median(iqr_delta_vec), quantile(iqr_delta_vec, 0.975)]]

In [ ]:
# Calculate summary statistics of parameter distributions numerically - using median values
mean_py0tilde = 0
mean_dy1tilde = 0
moment_2_py0tilde = 0
moment_2_dy1tilde = 0
for i = 1:N-1
    x_diff = py0tilde_lin[i+1]-py0tilde_lin[i]
    x_mean = mean([py0tilde_lin[i+1], py0tilde_lin[i]])
    y_mean = mean([pdf(truncated(Normal(med_bar_1, med_tau_1), lower=0),
                        f_inv_py0(py0tilde_lin[i+1],45/7)).*abs.(f_inv_py0_diff(py0tilde_lin[i+1],45/7)),
                 pdf(truncated(Normal(med_bar_1, med_tau_1), lower=0),
                        f_inv_py0(py0tilde_lin[i],45/7)).*abs.(f_inv_py0_diff(py0tilde_lin[i],45/7))])
    mean_py0tilde = mean_py0tilde + x_diff*y_mean*x_mean
    moment_2_py0tilde = moment_2_py0tilde + x_diff*y_mean*x_mean^2
    x_diff = dy1tilde_lin[i+1]-dy1tilde_lin[i]
    x_mean = mean([dy1tilde_lin[i+1], dy1tilde_lin[i]])
    y_mean = mean([pdf(truncated(Normal(med_bar_2, med_tau_2), lower=0),
                        f_inv_dy1(dy1tilde_lin[i+1],45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin[i+1],45/7)),
                 pdf(truncated(Normal(med_bar_2, med_tau_2), lower=0),
                        f_inv_dy1(dy1tilde_lin[i],45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin[i],45/7))])
    mean_dy1tilde = mean_dy1tilde + x_diff*y_mean*x_mean
    moment_2_dy1tilde = moment_2_dy1tilde + x_diff*y_mean*x_mean^2
end
var_py0tilde = moment_2_py0tilde-mean_py0tilde^2
var_dy1tilde = moment_2_dy1tilde-mean_dy1tilde^2
sd_py0tilde = sqrt(var_py0tilde)
sd_dy1tilde = sqrt(var_dy1tilde)

In [ ]:
# Storage
mean_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
mean_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
med_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
med_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
mode_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
mode_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
std_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
std_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
iqr_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
iqr_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
Q1_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
Q1_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
Q3_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
Q3_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
CV_py0tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
CV_dy1tilde_vec = zeros(length(df_MCMC[:,"iteration"]))
temp_pdf_py0tilde_vec = zeros(N-1)
temp_pdf_dy1tilde_vec = zeros(N-1)

# Calculate summary statistics of parameter distributions numerically - using entire chain
@showprogress dt=1 desc="Calculating summary statistics for all samples..." for j=1:length(df_MCMC[:,"iteration"])
    # Set values to 0 initially
    mean_py0tilde = 0
    mean_dy1tilde = 0
    moment_2_py0tilde = 0
    moment_2_dy1tilde = 0
    
    # Calculate statistics in loop
    for i = 1:N-1
        x_diff = py0tilde_lin[i+1]-py0tilde_lin[i]
        x_mean = mean([py0tilde_lin[i+1], py0tilde_lin[i]])
        y_mean = mean([pdf(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0),
                            f_inv_py0(py0tilde_lin[i+1],45/7)).*abs.(f_inv_py0_diff(py0tilde_lin[i+1],45/7)),
                     pdf(truncated(Normal(df_MCMC[j,"rhobar[1]"], df_MCMC[j,"rhotau[1]"]), lower=0),
                            f_inv_py0(py0tilde_lin[i],45/7)).*abs.(f_inv_py0_diff(py0tilde_lin[i],45/7))])
        mean_py0tilde = mean_py0tilde + x_diff*y_mean*x_mean
        moment_2_py0tilde = moment_2_py0tilde + x_diff*y_mean*x_mean^2
        temp_pdf_py0tilde_vec[i] = y_mean
        x_diff = dy1tilde_lin[i+1]-dy1tilde_lin[i]
        x_mean = mean([dy1tilde_lin[i+1], dy1tilde_lin[i]])
        y_mean = mean([pdf(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0),
                            f_inv_dy1(dy1tilde_lin[i+1],45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin[i+1],45/7)),
                     pdf(truncated(Normal(df_MCMC[j,"rhobar[2]"], df_MCMC[j,"rhotau[2]"]), lower=0),
                            f_inv_dy1(dy1tilde_lin[i],45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin[i],45/7))])
        mean_dy1tilde = mean_dy1tilde + x_diff*y_mean*x_mean
        moment_2_dy1tilde = moment_2_dy1tilde + x_diff*y_mean*x_mean^2
        temp_pdf_dy1tilde_vec[i] = y_mean
    end
    
    # Save results
    mean_py0tilde_vec[j] = mean_py0tilde
    mean_dy1tilde_vec[j] = mean_dy1tilde
    var_py0tilde = moment_2_py0tilde-mean_py0tilde^2
    var_dy1tilde = moment_2_dy1tilde-mean_dy1tilde^2
    sd_py0tilde = sqrt(var_py0tilde)
    sd_dy1tilde = sqrt(var_dy1tilde)
    std_py0tilde_vec[j] = sd_py0tilde
    std_dy1tilde_vec[j] = sd_dy1tilde
    mode_py0tilde_vec[j] = py0tilde_lin[argmax(temp_pdf_py0tilde_vec)]
    mode_dy1tilde_vec[j] = dy1tilde_lin[argmax(temp_pdf_dy1tilde_vec)]
    temp = cumsum(temp_pdf_py0tilde_vec.*(py0tilde_lin[2]-py0tilde_lin[1]))
    med_py0tilde_vec[j] = py0tilde_lin[findfirst(temp.>=0.5)]
    Q1_py0tilde_vec[j] = py0tilde_lin[findfirst(temp.>=0.25)]
    Q3_py0tilde_vec[j] = py0tilde_lin[findfirst(temp.>=0.75)]
    iqr_py0tilde_vec[j] = py0tilde_lin[findfirst(temp.>=0.75)]-py0tilde_lin[findfirst(temp.>=0.25)]
    temp = cumsum(temp_pdf_dy1tilde_vec.*(dy1tilde_lin[2]-dy1tilde_lin[1]))
    med_dy1tilde_vec[j] = dy1tilde_lin[findfirst(temp.>=0.5)]
    iqr_dy1tilde_vec[j] = dy1tilde_lin[findfirst(temp.>=0.75)]-dy1tilde_lin[findfirst(temp.>=0.25)]
    Q1_dy1tilde_vec[j] = dy1tilde_lin[findfirst(temp.>=0.25)]
    Q3_dy1tilde_vec[j] = dy1tilde_lin[findfirst(temp.>=0.75)]
    CV_py0tilde_vec[j] = std_py0tilde_vec[j]/mean_py0tilde_vec[j]
    CV_dy1tilde_vec[j] = std_dy1tilde_vec[j]/mean_dy1tilde_vec[j]
end

In [ ]:
[[quantile(mean_py0tilde_vec, 0.025), median(mean_py0tilde_vec), quantile(mean_py0tilde_vec, 0.975)],
 [quantile(mean_dy1tilde_vec, 0.025), median(mean_dy1tilde_vec), quantile(mean_dy1tilde_vec, 0.975)],
 [quantile(med_py0tilde_vec, 0.025), median(med_py0tilde_vec), quantile(med_py0tilde_vec, 0.975)],
 [quantile(med_dy1tilde_vec, 0.025), median(med_dy1tilde_vec), quantile(med_dy1tilde_vec, 0.975)],
 [quantile(mode_py0tilde_vec, 0.025), median(mode_py0tilde_vec), quantile(mode_py0tilde_vec, 0.975)],
 [quantile(mode_dy1tilde_vec, 0.025), median(mode_dy1tilde_vec), quantile(mode_dy1tilde_vec, 0.975)],
 [quantile(std_py0tilde_vec, 0.025), median(std_py0tilde_vec), quantile(std_py0tilde_vec, 0.975)],
 [quantile(std_dy1tilde_vec, 0.025), median(std_dy1tilde_vec), quantile(std_dy1tilde_vec, 0.975)],
 [quantile(iqr_py0tilde_vec, 0.025), median(iqr_py0tilde_vec), quantile(iqr_py0tilde_vec, 0.975)],
 [quantile(iqr_dy1tilde_vec, 0.025), median(iqr_dy1tilde_vec), quantile(iqr_dy1tilde_vec, 0.975)],
 [quantile(CV_py0tilde_vec, 0.025), median(CV_py0tilde_vec), quantile(CV_py0tilde_vec, 0.975)],
 [quantile(CV_dy1tilde_vec, 0.025), median(CV_dy1tilde_vec), quantile(CV_dy1tilde_vec, 0.975)]]

In [ ]:
median(iqr_dy1tilde_vec)/p.dy1

In [ ]:
[median(Q1_dy1tilde_vec), median(Q3_dy1tilde_vec)]

In [ ]:
[median(Q3_dy1tilde_vec)/p.dy1,median(Q1_dy1tilde_vec)/p.dy1]

In [ ]:
median(Q3_dy1tilde_vec)/p.dy1 - median(Q1_dy1tilde_vec)/p.dy1

In [ ]:
1-0.394/p.py0

In [ ]:
1-0.481/p.py0

In [ ]:
[median(Q1_py0tilde_vec), median(Q3_py0tilde_vec)]

In [ ]:
[(1-median(Q3_py0tilde_vec))/p.py0,(1-median(Q1_py0tilde_vec))/p.py0]

In [ ]:
(1-median(Q1_py0tilde_vec))/p.py0-(1-median(Q3_py0tilde_vec))/p.py0

In [ ]:
# Plot population distributions - with histograms
l = @layout [a; a;]
plot_effect1 = histogram(med_param[:,1], bins=range(0,1,21), label = "", normalize = :pdf)
plot!(truncated(Normal(med_bar_1, med_tau_1), lower=0), linewidth = 3, colour = :red, label = "")
plot!(rho_lin, quantile.(eachrow(pdf_rho_mat), 0.025), fillrange = quantile.(eachrow(pdf_rho_mat), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = "", linealpha = 0)
xlims!(0.0,1)
ylims!(0.0,Inf)
title!("\nPopulation Distribution for "*L"\rho"*"\n" *L"\mu_{\rho} = "*
        "$(round(med_bar_1; sigdigits=3)), "*
        L"\sigma_{\rho} = "*"$(round(med_tau_1; sigdigits=3))")
xlabel!(L"\rho/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
plot_effect2 = histogram(med_param[:,2], bins=range(0,1,21), label = "", normalize = :pdf)
plot!(truncated(Normal(med_bar_2, med_tau_2), lower=0), 
                    linewidth = 3, colour = :red, label = "")
plot!(delta_lin, quantile.(eachrow(pdf_delta_mat), 0.025), fillrange = quantile.(eachrow(pdf_delta_mat), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = "", linealpha = 0)
title!("\nPopulation Distribution for "*L"\delta"*"\n" *L"\mu_{\delta} = "*
        "$(round(med_bar_2; sigdigits=3)), "*
        L"\sigma_{\delta} = "*"$(round(med_tau_2; sigdigits=3))")
xlims!(0.0,1.0)
ylims!(0.0,Inf)
xlabel!(L"\delta/\textrm{(\mu g/day)}^{-1}")
ylabel!("pdf/1")
figcombined1 = plot(plot_effect1,plot_effect2,layout = l,size = (700, 1000), margin=5mm)

# Plot distributions of estimated parameters given standard dose
# Plot population distributions - with histograms
l = @layout [a; a;]
plot_effect1 = histogram(med_py0, bins=range(0,1,21), label = "", normalize = :pdf)
# plot!(truncated(Normal(med_bar_1, med_tau_1), lower=0), linewidth = 3, colour = :red, label = "")
plot!(py0tilde_lin, pdf(truncated(Normal(med_bar_1, med_tau_1), lower=0),
                        f_inv_py0(py0tilde_lin,45/7)).*abs.(f_inv_py0_diff(py0tilde_lin,45/7)), 
            linewidth = 3, colour = :red, label = "")
plot!(py0tilde_lin, quantile.(eachrow(pdf_py0_mat), 0.025), fillrange = quantile.(eachrow(pdf_py0_mat), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = "", linealpha = 0)
plot!([p.py0,p.py0],[0,1000], linewidth = 3, colour = :green1, label = "", linestyle = :dash)
xlims!(0.0,1)
# ylims!(0.0,3.5)
title!("\nPopulation Distribution for "*L"\widetilde{p}_{y_0}"*"\n"
        *L"assuming constant IFN-$\alpha$ dosing of $45\ \mu\mathrm{g/week}$"*"\n"
        *L"\mathrm{med}(\widetilde{p}_{y_0}) = "*
        "$(round(median(med_py0tilde_vec); sigdigits=3)), "*
        "std"*L"(\widetilde{p}_{y_0}) = "*"$(round(median(std_py0tilde_vec); sigdigits=3))")
xlabel!(L"\widetilde{p}_{y_0}/1")
ylabel!("pdf/1")
plot_effect2 = histogram(med_dy1, bins=range(0,0.025,21), label = "", normalize = :pdf)
# plot!(truncated(Normal(med_bar_2, med_tau_2), lower=0), 
#                     linewidth = 3, colour = :red, label = "")
plot!(dy1tilde_lin, pdf(truncated(Normal(med_bar_2, med_tau_2), lower=0),
                        f_inv_dy1(dy1tilde_lin,45/7)).*abs.(f_inv_dy1_diff(dy1tilde_lin,45/7)), 
            linewidth = 3, colour = :red, label = "")
plot!(dy1tilde_lin, quantile.(eachrow(pdf_dy1_mat), 0.025), fillrange = quantile.(eachrow(pdf_dy1_mat), 0.975), fillalpha = 0.35, 
         colour = :grey60, label = "", linealpha = 0)
plot!([p.dy1,p.dy1],[0,1000], linewidth = 3, colour = :green1, label = "", linestyle = :dash)
title!("\nPopulation Distribution for "*L"\widetilde{d}_{y_1}"*"\n" 
        *L"assuming constant IFN-$\alpha$ dosing of $45\ \mu\mathrm{g/week}$"*"\n"
        *L"\mathrm{med}(\widetilde{d}_{y_1}) = "*
        "$(round(median(med_dy1tilde_vec); sigdigits=3)), "*
        "std"*L"(\widetilde{d}_{y_1}) = "*"$(round(median(std_dy1tilde_vec); sigdigits=3))")
xlims!(0.0,0.025)
# ylims!(0.0,160)
xlabel!(L"\widetilde{d}_{y_1}/\textrm{day}^{-1}")
ylabel!("pdf/1")
figcombined2 = plot(plot_effect1,plot_effect2,layout = l,size = (700, 1000), margin=5mm)

# Plot together
figcombined = plot(figcombined1,figcombined2,size = (1000, 1000), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figcombined,figpath*"Popluation_distribution_double_$(effect).png")
savefig(figcombined,figpath*"Popluation_distribution_double_$(effect).pdf")
savefig(figcombined,figpath*"Popluation_distribution_double_$(effect).svg")

## Compare results to true values

In [ ]:
# Compare values
fig_rho_comparison = scatter(df_param.patientID,df_param.rho, label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!(df_param.patientID,med_param[:,1], label="Estimated (median) valaues", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!(L"\rho")
xlabel!("Patient ID")
ylabel!(L"\rho/\textrm{(\mu g/day)}^{-1}")
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :outerbottom)
fig_delta_comparison = scatter(df_param.patientID,df_param.delta, label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!(df_param.patientID,med_param[:,2], label="Estimated (median) valaues", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!(L"\delta")
xlabel!("Patient ID")
ylabel!(L"\delta/\textrm{(\mu g/day)}^{-1}")
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :outerbottom)
fig_initJAK_comparison = scatter(df_param.patientID,df_param.initJAK, label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!(df_param.patientID,med_initJAK, label="Estimated (median) valaues", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!(L"v_0")
xlabel!("Patient ID")
ylabel!(L"$v_0$/%")
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :outerbottom)
fig_kL_comparison = scatter(df_param.patientID,df_param.kL, label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!(df_param.patientID,k_WBC_vec, label="Estimated valaues", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!(L"k_\textrm{L}")
xlabel!("Patient ID")
ylabel!(L"$k_\textrm{L}$/1")
fig_kT_comparison = scatter(df_param.patientID,df_param.kT, label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!(df_param.patientID,k_TRC_vec, label="Estimated valaues", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!(L"k_\textrm{T}")
xlabel!("Patient ID")
ylabel!(L"$k_\textrm{T}$/1")
variable_names = [L"\mu_{\rho}", L"\mu_{\delta}", L"\sigma_{\rho}", L"\sigma_{\delta}"]
fig_population_comparison = scatter([1], [rho_bar], label="True values", markercolor=:red, markersize=6, markerstrokewidth=0.25,
                            xticks = (1:4, variable_names))
scatter!([1], [med_bar_1], label = "Estimated (median) values", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
scatter!([2], [delta_bar], label="", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!([2], [med_bar_2], label = "", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
scatter!([3], [rho_sd], label="", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!([3], [med_tau_1], label = "", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
scatter!([4], [delta_sd], label="", markercolor=:red, markersize=6, markerstrokewidth=0.25)
scatter!([4], [med_tau_2], label = "", markercolor=:blue, markersize=6, markerstrokewidth=0.25,
         markershape=:rect)
title!("Population parameters")
xlabel!("Parameter")
ylabel!(L"Value/$\textrm{(\mu g/day)}^{-1}$")
xlims!(0,5)
ylims!(0,0.4)
fig_comparison = plot(fig_rho_comparison, fig_delta_comparison, fig_initJAK_comparison, fig_kL_comparison, fig_kT_comparison,
     fig_population_comparison, layout= (2,3), size = (2000, 1000), margin=8mm, leftmargin = 10mm, rightmargin = 8mm)
plot!(titlefont=24,xtickfontsize=20,ytickfontsize=20,xguidefontsize=20,yguidefontsize=20,legendfontsize=15, 
        legend = :outertop)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_comparison,figpath*"fig_comparison.png")
savefig(fig_comparison,figpath*"fig_comparison.pdf")
savefig(fig_comparison,figpath*"fig_comparison.svg")

## Chain diagnostics

In [ ]:
# Convert dataframe to chain object
chain = Chains(Array(df_MCMC[:,3:end-1]), names(df_MCMC)[3:end-1])

In [ ]:
# Calculate diagnostics
diagnostics = ess_rhat(chain)[:,:]

In [ ]:
# Calculate quantities of interest
sum(diagnostics[:,2].<1.01)

In [ ]:
maximum(diagnostics[:,2])

In [ ]:
sum(diagnostics[:,2].<1.01)/size(diagnostics,1)

In [ ]:
(sum(diagnostics[:,2].<1.01)+8)/size(diagnostics,1)

In [ ]:
ID = diagnostics[:,2].>1.01

In [ ]:
findall(==(1),ID)

In [ ]:
mean(diagnostics[:,1])

In [ ]:
median(diagnostics[:,1])

In [ ]:
quantile(diagnostics[:,1],0.025)

In [ ]:
minimum(diagnostics[:,1])

In [ ]:
# Plot convergence dianogstis
l = @layout [a a; a a]
plot_ess_variables = plot(diagnostics[:,1],linewidth = 0.5, colour = 1, label = "")
title!("\nEffective Sample Size")
xlabel!("Variable")
ylabel!("ESS/1")
plot_ess_histogram = plot(histogram(diagnostics[:,1]), colour = 1, legend = false)
title!("\nHistogram for Effective\n Sample Size")
xlabel!("ESS/1")
ylabel!("Count/1")
plot_rhat_variables = plot(diagnostics[:,2],linewidth = 0.5, colour = 1, label = "")
title!("\n"*L"\hat{R}")
xlabel!("Variable")
ylabel!(L"\hat{R}/1")
ylims!(0.99,1.05)
plot_rhat_histogram = plot(histogram(diagnostics[:,2]), colour = 1, legend = false)
title!("\nHistogram for "*L"\hat{R}")
xlabel!(L"\hat{R}/1")
ylabel!("Count/1")
figDiagnostics = plot(plot_ess_variables,plot_ess_histogram,plot_rhat_variables,plot_rhat_histogram,
                      layout = l,size = (700, 1000), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figDiagnostics,figpath*"Hierarchical_chain_diagnostics_$(effect).png")
savefig(figDiagnostics,figpath*"Hierarchical_chain_diagnostics_$(effect).pdf")
savefig(figDiagnostics,figpath*"Hierarchical_chain_diagnostics_$(effect).svg")

## Calculate dose-response relationships

In [ ]:
# Define doses we want to investigate we want to investigate
cI_lin = range(0,180,51)./7

# Choose patients
p_vec = [2,3,4,5]

# Storage
R_factor_mat = zeros(length(cI_lin),length(df_MCMC[:,"iteration"]),length(p_vec))
VAF_mat_const = zeros(length(pDays_const), length(df_MCMC[:,"iteration"]))

@showprogress dt=1 desc="Calculating dose-response relationship..." for i=1:length(cI_lin)
    for j=1:length(p_vec)
        # Extract relevant data
        pID = p_vec[j]
        df_p = df_D[df_D.patientID .== pID, :]

        # Create copy of df_p with constant dose 
        df_p_const = copy(df_p)
        df_p_const.IFN[df_p_const.days.>=0] .= cI_lin[i]

        # Extract VAF
        pVAF = df_p.JAK

        # Choose treatment effect
        effect = "py0dy1"

        # Choose time for calculating VAF
        pDays_const = [0,2*365]

        # Load standard parameters
        include("model_default_param.jl")

        # Calculate VAF in loop
        for k in eachindex(df_MCMC[:,"iteration"])
            # Set rho values
            rho = [df_MCMC[k,"theta_$(pID)[1]"], df_MCMC[k,"theta_$(pID)[2]"]]

            # Set initial JAK
            initJAK = df_MCMC[k,"theta_$(pID)[3]"]

            # Calculate VAF using function
            VAF_mat_const[:,k], sol_const = model_calc_VAF(rho,df_p_const,effect,pDays_const,p,master_curve,initJAK)
        end
    
        # Calculate R-factor and append
        R_factor_mat[i,:,j] = 100*(1 .- VAF_mat_const[end,:]./df_MCMC[:,"theta_$(pID)[3]"])
    end
end

In [ ]:
fig_R_factor = plot()
for i=1:length(p_vec)
    plot!(cI_lin*7,quantile.(eachrow(R_factor_mat[:,:,i]),0.025), fillrange = quantile.(eachrow(R_factor_mat[:,:,1]),0.975), 
            fillalpha = 0.35, colour = i, label = "", linealpha = 0)
end
for i=1:length(p_vec)
    plot!(cI_lin*7,median(R_factor_mat[:,:,i],dims=2), linewidth = 6, label = "Patient $(p_vec[i])", colour= i,
            xticks = (0:45:180, 0:45:180))
end
title!(L"$R$-Factor as a function of IFN-$\alpha$" *"\n dose for 4 representative patients")
xlabel!(L"IFN-$\alpha$/$\mathrm{\frac{\mu g}{week}}$")
ylabel!(L"$R$-Factor/%")
xlims!(0, 180)
ylims!(-30, 100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, margin = 5mm,
legend = :right)

In [ ]:
# Number of samples
n_samples = 10500

# Number of patients
P = length(unique_patients)

# Storage
all_param = Array{Float64, 2}(undef, P*(n_samples-500), 2)

# Extract mean parameter values
for i=1:P 
    # Set pID
    pID = unique_patients[i]
    
    # Extract data
    all_param[(i-1)*(n_samples-500)+1:i*(n_samples-500),1] = df_MCMC[:,"theta_$(pID)[1]"]
    all_param[(i-1)*(n_samples-500)+1:i*(n_samples-500),2] = df_MCMC[:,"theta_$(pID)[2]"]
end

In [ ]:
# Generate samples
Q = 10^4
rand_ID = rand(1:size(all_param,1),Q)

# Bootstrap
# bootstrap_param = all_param[rand_ID,:]
bootstrap_param = all_param

# Define days for storing VAF
pDays = 0:30:2*365

# Define initVAF
initJAK = 0.38

# Define doses we want to investigate we want to investigate
cI_lin = (0:3:180)./7

# Storage
R_factor_mat_all = zeros(length(cI_lin),size(bootstrap_param,1))

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF and R-factor for all doses" for i=1:length(cI_lin)
    for j=1:size(bootstrap_param,1)
        # Define artificial patient with constant dose of IFN
        df_p = DataFrame(patientID = [-1,-1], days = [0,730], IFN = [cI_lin[i],cI_lin[i]], RUX = [NaN, NaN], JAK = [NaN, NaN], 
                         TRC = [NaN, NaN], WBC = [NaN, NaN])

        # Define parameters
        rho = bootstrap_param[j,:]

        # Calculate VAF using function
        VAF_mat_art, sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)

        # Calculate R-factor and append
        R_factor_mat_all[i,j] = 100*(1 .- VAF_mat_art[end]./initJAK)
    end
end

In [ ]:
fig_R_factor_all = plot()
plot!(cI_lin*7,quantile.(eachrow(R_factor_mat_all),0.025), fillrange = quantile.(eachrow(R_factor_mat_all),0.975), 
            fillalpha = 0.35, colour = :grey60, label = "", linealpha = 0)
plot!(cI_lin*7,quantile.(eachrow(R_factor_mat_all),0.125), fillrange = quantile.(eachrow(R_factor_mat_all),0.875), 
            fillalpha = 0.35, colour = :grey40, label = "", linealpha = 0)
plot!(cI_lin*7,quantile.(eachrow(R_factor_mat_all),0.25), fillrange = quantile.(eachrow(R_factor_mat_all),0.75), 
            fillalpha = 0.35, colour = :grey20, label = "", linealpha = 0)
plot!(cI_lin*7,median(R_factor_mat_all,dims=2), linewidth = 6, label = "", colour= :blue1,
        xticks = (0:45:180, 0:45:180))
title!(L"$R$-Factor as a function of IFN-$\alpha$" *"\n dose for the population")
xlabel!(L"IFN-$\alpha$/$\mathrm{\frac{\mu g}{week}}$")
ylabel!(L"$R$-Factor/%")
xlims!(0, 180)
ylims!(-30, 100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, margin = 5mm)

In [ ]:
fig_dose_response = plot(fig_R_factor,fig_R_factor_all, layout = grid(1,2), size = (1400, 700), margin=5mm, leftmargin = 8mm, rightmargin = 8mm,
     bottommargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_dose_response,figpath*"dose_response.png")
savefig(fig_dose_response,figpath*"dose_response.pdf")
savefig(fig_dose_response,figpath*"dose_response.svg")

## $R$-factor for all patients

In [ ]:
# Unique patients
unique_patients = unique(df_D.patientID)

# Number of patients
P = length(unique_patients)

# Storage
mean_param = Array{Float64, 2}(undef, P, 2)
median_param = Array{Float64, 2}(undef, P, 2)

# Extract mean parameter values
for i=1:P
    # Set pID
    pID = unique_patients[i]
    
    # Extract data
    mean_param[i,1] = mean(df_MCMC[:,"theta_$(pID)[1]"])
    mean_param[i,2] = mean(df_MCMC[:,"theta_$(pID)[2]"])
    median_param[i,1] = median(df_MCMC[:,"theta_$(pID)[1]"])
    median_param[i,2] = median(df_MCMC[:,"theta_$(pID)[2]"])
end

# Define mean and median parameter vectors
mean_param_1_vec = mean_param[:,1]
mean_param_2_vec = mean_param[:,2]
median_param_1_vec = median_param[:,1]
median_param_2_vec = median_param[:,2]

In [ ]:
fig_R_factor = violin(collect(1:P)', R_factor', legend = false, xticks = (1:P, unique_patients))
scatter!(collect(1:P),median(R_factor[:,:],dims=2), markershape = :hline,color = :black, markersize = 4)
title!("Distribution of R-Factor for all Patients")
xlabel!("Patient ID")
ylabel!("R/"*L"\%")
ylims!(-50,100)
plot!(titlefont=15,xtickfontsize=6,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, 
        legend = :none, size = (1200, 500), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_R_factor,figpath*"R_factor.png")
savefig(fig_R_factor,figpath*"R_factor.pdf")
savefig(fig_R_factor,figpath*"R_factor.svg")

In [ ]:
# Parameter correlation - R-factor
fig_R_factor = scatter(median_param_1_vec,median_param_2_vec, zcolor=mean(R_factor[:,:],dims=2), 
                            markersize=6, label = "", color = :rainbow)
title!("R-Factor")
xlabel!(L"\rho/\mathrm{(\frac{\mu g}{day}})^{-1}")
ylabel!(L"\delta/\mathrm{(\frac{\mu g}{day}})^{-1}")
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_R_factor,figpath*"R_factor.png")
savefig(fig_R_factor,figpath*"R_factor.pdf")
savefig(fig_R_factor,figpath*"R_factor.svg")

## Population response to treatment schedules

In [ ]:
# Define virtual patients with varying IFN doses
df_p_1 = DataFrame(patientID = [-1,-1], days = [0,730], IFN = [45/7,45/7], RUX = [NaN, NaN], 
                   JAK = [NaN, NaN], TRC = [NaN, NaN], WBC = [NaN, NaN])
df_p_2 = DataFrame(patientID = [-1,-1,-1], days = [0,365,730], IFN = [45/7,45/14,45/28], RUX = [NaN, NaN, NaN], 
                   JAK = [NaN, NaN, NaN], TRC = [NaN, NaN, NaN], WBC = [NaN, NaN, NaN])
df_p_3 = DataFrame(patientID = [-1,-1,-1], days = [0,365,730], IFN = [45/7,67.5/7,90/7], RUX = [NaN, NaN, NaN], 
                   JAK = [NaN, NaN, NaN], TRC = [NaN, NaN, NaN], WBC = [NaN, NaN, NaN])
df_p_4 = DataFrame(patientID = [-1,-1], days = [0,730], IFN = [90/7, 90/7], RUX = [NaN, NaN], 
                   JAK = [NaN, NaN], TRC = [NaN, NaN], WBC = [NaN, NaN])

# Define days for storing VAF
pDays = 0:30.4:7*365

# Choose seed
Random.seed!(42)

# Generate samples
Q = 10^4
rand_ID = rand(1:size(all_param,1),Q)

# Bootstrap
# bootstrap_param = all_param[rand_ID,:]
bootstrap_param = all_param

# Storage 
VAF_gen_1 = zeros(length(pDays), size(bootstrap_param,1))
VAF_gen_2 = zeros(length(pDays), size(bootstrap_param,1))
VAF_gen_3 = zeros(length(pDays), size(bootstrap_param,1))
VAF_gen_4 = zeros(length(pDays), size(bootstrap_param,1))

# Calculate VAF in loop
@showprogress dt=1 desc="Calculating VAF for all parameter samples" for i = 1:size(VAF_gen_1,2)
    # Set rho values
    rho = [bootstrap_param[i,1], bootstrap_param[i,2]]

    # Set initial JAK
    initJAK = 0.38

    # Calculate VAF using function
    VAF_gen_1[:,i], sol = model_calc_VAF(rho,df_p_1,effect,pDays,p,master_curve,initJAK)
    VAF_gen_2[:,i], sol = model_calc_VAF(rho,df_p_2,effect,pDays,p,master_curve,initJAK)
    VAF_gen_3[:,i], sol = model_calc_VAF(rho,df_p_3,effect,pDays,p,master_curve,initJAK)
    VAF_gen_4[:,i], sol = model_calc_VAF(rho,df_p_4,effect,pDays,p,master_curve,initJAK)
end

In [ ]:
for i=1:4
    if i==1
        maxDays = maximum(df_p_1.days)
        figVAF = plot(df_p_1.days/30.4, zeros(size(df_p_1.days)), fillrange = df_p_1.IFN, line =:steppost, fillalpha = 0.1, 
                      linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p_1.IFN[end],df_p_1.IFN[end]], line =:steppost, fillalpha = 0.1, 
               linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        title!("Predicted population " * L"JAK2"*" VAF response \nto a constant low dose of "* L"IFN-$\alpha$")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)
        p2 = twinx()
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_1), 0.025)*100, fillrange = quantile.(eachrow(VAF_gen_1), 0.975)*100, 
              fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_1), 0.125)*100, fillrange = quantile.(eachrow(VAF_gen_1), 0.875)*100, 
              fillalpha = 0.35, colour = :grey40, label = L"\textrm{90\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_1), 0.25)*100, fillrange = quantile.(eachrow(VAF_gen_1), 0.75)*100, 
              fillalpha = 0.35, colour = :grey20, label = L"\textrm{50\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, median(VAF_gen_1,dims=2)*100, linewidth = 6, label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    elseif i==2
        maxDays = maximum(df_p_2.days)
        figVAF = plot(df_p_2.days/30.4, zeros(size(df_p_2.days)), fillrange = df_p_2.IFN, line =:steppost, fillalpha = 0.1, 
                      linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p_2.IFN[end],df_p_2.IFN[end]], line =:steppost, fillalpha = 0.1, 
               linealpha = 0, linewidth = 6, label = "", colour= :blue1)
        title!("Predicted population " * L"JAK2"*" VAF response \nto a decreasing dose of "* L"IFN-$\alpha$")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)
        p2 = twinx()
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_2), 0.025)*100, fillrange = quantile.(eachrow(VAF_gen_2), 0.975)*100, 
              fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_2), 0.125)*100, fillrange = quantile.(eachrow(VAF_gen_2), 0.875)*100, 
              fillalpha = 0.35, colour = :grey40, label = L"\textrm{90\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_2), 0.25)*100, fillrange = quantile.(eachrow(VAF_gen_2), 0.75)*100, 
              fillalpha = 0.35, colour = :grey20, label = L"\textrm{50\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, median(VAF_gen_2,dims=2)*100, linewidth = 6, label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    elseif i==3
        maxDays = maximum(df_p_3.days)
        figVAF = plot(df_p_3.days/30.4, zeros(size(df_p_3.days)), fillrange = df_p_3.IFN, line =:steppost, fillalpha = 0.1, 
                      linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p_3.IFN[end],df_p_3.IFN[end]], line =:steppost, fillalpha = 0.1, 
               linealpha = 0, linewidth = 6, label = "", colour= :blue1)
        title!("Predicted population " * L"JAK2"*" VAF response \nto an increasing dose of "* L"IFN-$\alpha$")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)
        p2 = twinx()
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_3), 0.025)*100, fillrange = quantile.(eachrow(VAF_gen_3), 0.975)*100, 
              fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_3), 0.125)*100, fillrange = quantile.(eachrow(VAF_gen_3), 0.875)*100, 
              fillalpha = 0.35, colour = :grey40, label = L"\textrm{90\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_3), 0.25)*100, fillrange = quantile.(eachrow(VAF_gen_3), 0.75)*100, 
              fillalpha = 0.35, colour = :grey20, label = L"\textrm{50\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, median(VAF_gen_3,dims=2)*100, linewidth = 6, label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    elseif i==4
        maxDays = maximum(df_p_4.days)
        figVAF = plot(df_p_4.days/30.4, zeros(size(df_p_4.days)), fillrange = df_p_4.IFN, line =:steppost, fillalpha = 0.1, 
                      linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p_4.IFN[end],df_p_4.IFN[end]], line =:steppost, fillalpha = 0.1, 
               linealpha = 0, linewidth = 3, label = "", colour= :blue1)
        title!("Predicted population " * L"JAK2"*" VAF response \nto a constant higher dose of "* L"IFN-$\alpha$")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)
        p2 = twinx()
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_4), 0.025)*100, fillrange = quantile.(eachrow(VAF_gen_4), 0.975)*100, 
              fillalpha = 0.35, colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_4), 0.125)*100, fillrange = quantile.(eachrow(VAF_gen_4), 0.875)*100, 
              fillalpha = 0.35, colour = :grey40, label = L"\textrm{90\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, quantile.(eachrow(VAF_gen_4), 0.25)*100, fillrange = quantile.(eachrow(VAF_gen_4), 0.75)*100, 
              fillalpha = 0.35, colour = :grey20, label = L"\textrm{50\%\ Credibility\ Interval}", linealpha = 0)
        plot!(p2, pDays/30.4, median(VAF_gen_4,dims=2)*100, linewidth = 6, label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none)
    end

    if i==1
        fig1 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==2
        fig2 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==3
        fig3 = plot(figVAF, size = (700, 500), margin=5mm)
    elseif i==4
        fig4 = plot(figVAF, size = (700, 500), margin=5mm)
    end
end

# Plot all 4 figures
figCombined = plot(fig1, fig2, fig3, fig4, layout=grid(2,2), size = (1500, 1000), margin = 5mm, leftmargin = 8mm, 
                   rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(figCombined,figpath*"VAF_gen.png")
savefig(figCombined,figpath*"VAF_gen.pdf")
savefig(figCombined,figpath*"VAF_gen.svg")

In [ ]:
print([quantile(VAF_gen_1[2*12+1,:],0.025), median(VAF_gen_1[2*12+1,:]), quantile(VAF_gen_1[2*12+1,:],0.975)])
print([quantile(VAF_gen_2[2*12+1,:],0.025), median(VAF_gen_2[2*12+1,:]), quantile(VAF_gen_2[2*12+1,:],0.975)])
print([quantile(VAF_gen_3[2*12+1,:],0.025), median(VAF_gen_3[2*12+1,:]), quantile(VAF_gen_3[2*12+1,:],0.975)])
print([quantile(VAF_gen_4[2*12+1,:],0.025), median(VAF_gen_4[2*12+1,:]), quantile(VAF_gen_4[2*12+1,:],0.975)])

In [ ]:
[sum(VAF_gen_1[2*12+1,:].<0.33)/size(VAF_gen_1[2*12+1,:],1), 
 sum(VAF_gen_2[2*12+1,:].<0.33)/size(VAF_gen_1[2*12+1,:],1), 
 sum(VAF_gen_3[2*12+1,:].<0.33)/size(VAF_gen_1[2*12+1,:],1),
 sum(VAF_gen_4[2*12+1,:].<0.33)/size(VAF_gen_1[2*12+1,:],1)]

## Measured vs. predicted VAF

In [ ]:
# Plot real vs. predicted VAF
plot_real_predict = scatter(real_VAF*100,predicted_VAF*100, markercolor=:red, markersize=4, markerstrokewidth=0.25, label = "")
plot!([0,100],[0,100], linewidth = 3, colour = :black, label="")
title!(L"Measured vs. Predicted $JAK2$ VAF")
xlabel!(L"Measured $JAK2$ VAF/%")
ylabel!(L"Predicted $JAK2$ VAF/%")
xlims!(0,100)
ylims!(0,100)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
fig_real_predict = plot(plot_real_predict, size = (1000, 700), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_real_predict,figpath*"Hierarchical_real_predict_$(effect).png")
savefig(fig_real_predict,figpath*"Hierarchical_real_predict_$(effect).pdf")
savefig(fig_real_predict,figpath*"Hierarchical_real_predict_$(effect).svg")

In [ ]:
# Plot predicted VAF vs. residuals
plot_predict_residual = scatter(predicted_VAF*100,(real_VAF-predicted_VAF)*100, markercolor=:red, markersize=4, markerstrokewidth=0.25, label = "")
title!(L"Residual vs. Inferred $JAK2$ VAF")
xlabel!(L"Inferred $JAK2$ VAF/%")
ylabel!(L"Residual $JAK2$ VAF/%")
xlims!(0,100)
ylims!(-60,60)
plot!(titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, 
        legend = :none)
fig_predict_residual = plot(plot_predict_residual, size = (1000, 700), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_predict_residual,figpath*"Hierarchical_predict_residual_$(effect).png")
savefig(fig_predict_residual,figpath*"Hierarchical_predict_residual_$(effect).pdf")
savefig(fig_predict_residual,figpath*"Hierarchical_predict_residual_$(effect).svg")

## Violin plots for parameters

In [ ]:
# Unique patients
unique_patients = unique(df_D.patientID)

# Extract rho_p_y_0
df_rho_p_y_0 = df_MCMC[:,7:4:end-1]
df_rho_p_y_0 = Matrix(df_rho_p_y_0)

fig_violin = violin(collect(1:P)', df_rho_p_y_0, legend = false, xticks = (1:P, unique_patients))
scatter!(collect(1:P)',median(df_rho_p_y_0,dims=1), markershape = :hline,color = :black, markersize = 4)
title!("\nDistribution of "*L"\rho "*" for all patients")
xlabel!("Patient ID")
ylabel!(L"\rho/\textrm{(\mu g/day)}^{-1}")
ylims!(0.0,1)
plot!(titlefont=15,xtickfontsize=6,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, 
        legend = :none, size = (1200, 500), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_violin,figpath*"violin_py0.png")
savefig(fig_violin,figpath*"violin_py0.pdf")
savefig(fig_violin,figpath*"violin_py0.svg")

In [ ]:
# Unique patients
unique_patients = unique(df_D.patientID)

# Extract rho_d_y_1
df_rho_d_y_1 = df_MCMC[:,8:4:end-1]
df_rho_d_y_1 = Matrix(df_rho_d_y_1)

fig_violin = violin(collect(1:P)', df_rho_d_y_1, legend = false, xticks = (1:P, unique_patients))
scatter!(collect(1:P)',median(df_rho_d_y_1,dims=1), markershape = :hline,color = :black, markersize = 4)
title!("\nDistribution of "*L"\delta "*" for all patients")
xlabel!("Patient ID")
ylabel!(L"\delta/\textrm{(\mu g/day)}^{-1}")
ylims!(0.0,1)
plot!(titlefont=15,xtickfontsize=6,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15, 
        legend = :none, size = (1200, 500), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/Pseudodata/"
savefig(fig_violin,figpath*"violin_dy1.png")
savefig(fig_violin,figpath*"violin_dy1.pdf")
savefig(fig_violin,figpath*"violin_dy1.svg")